<a href="https://colab.research.google.com/github/jwyangyin/TFM/blob/main/Sistema_3_TFM_(Master_Data_Science)_Sistema_de_recomendaci%C3%B3n_h%C3%ADbrido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="
  background:#f6f8fa;
  padding:30px;
  border:1px solid #d0d7de;
  border-radius:10px;
">
  <b style="font-size:18px;">Índice</b>
  <ul>
    <li><a href="#intro">1. Introducción</a></li>
    <li><a href="#fuentes">1.1. Fuentes de datos y estructura</a></li>
    <li><a href="#carga">1.2. Carga de datos</a></li>
    <li><a href="#contexto">1.3. Contexto de los datos y definición del objetivo</a></li>
    <li><a href="#estadisticas">1.4. Estadísticas básicas del subconjunto All_Beauty</a></li>
    <li><a href="#enfoque">2. Enfoque del sistema de recomendación híbrido</a></li>
    <li><a href="#intuicion">2.1. Intuición del enfoque híbrido</a></li>
    <li><a href="#representacion">2.2. Representación de los datos</a></li>
    <li><a href="#senales">2.3. Señales utilizadas en el sistema híbrido</a></li>
    <li><a href="#hibridacion">2.4. Estrategia de combinación (hibridación)</a></li>
    <li><a href="#recomendaciones">2.5. Generación de recomendaciones</a></li>
    <li><a href="#justificacion">2.6. Justificación del enfoque híbrido</a></li>
    <li><a href="#limitaciones">2.7. Limitaciones del enfoque híbrido</a></li>
    <li><a href="#cap3">3. Análisis exploratorio y diagnóstico del dataset para el sistema híbrido</a></li>
    <li><a href="#consistencia">3.1. Consistencia entre interacciones y metadatos</a></li>
    <li><a href="#diagnostico">3.2. Diagnóstico de la señal colaborativa</a></li>
    <li><a href="#diagnosticocontenido">3.3. Diagnóstico de la señal de contenido</a></li>
    <li><a href="#implicaciones">3.4. Implicaciones del diagnóstico para el sistema de recomendación híbrido</a></li>
    <li><a href="#cap4">4. Preparación de datos para el sistema de recomendación híbrido</a></li>
    <li><a href="#definicion">4.1. Definición de la señal de interacción</a></li>
    <li><a href="#filtradoconsistencia">4.2. Filtrado básico de consistencia</a></li>
    <li><a href="#filtradofrecuencia">4.3. Filtrado por frecuencia para estabilidad del sistema híbrido</a></li>
    <li><a href="#indexado">4.4. Indexado de usuarios e ítems y construcción de la matriz CSR</a></li>
    <li><a href="#disenoeimplementacion">5. Diseño e implementación del sistema de recomendación híbrido</a></li>
    <li><a href="#estrategia">5.1. Estrategia de hibridación</a></li>
    <li><a href="#implementacioncolaborativo">5.2. Implementación del componente colaborativo (Item-to-Item)</a></li>
    <li><a href="#implementacioncontenido">5.3. Implementación del componente basado en contenido (Content-Based)</a></li>
    <li><a href="#cap5_4">5.4. Combinación de recomendaciones (hibridación tardía)</a></li>
    <li><a href="#cap5_5">5.5. Síntesis del diseño e implementación del sistema de recomendación híbrido</a></li>
    <li><a href="#cap6">6. Evaluación del sistema de recomendación híbrido</a></li>
    <li><a href="#cap6_1">6.1. Objetivos de la evaluación</a></li>
    <li><a href="#cap6_2">6.2. Protocolo experimental</a></li>
    <li><a href="#cap6_3">6.3. Métricas de evaluación</a></li>
    <li><a href="#cap6_4">6.4. Resultados experimentales</a></li>
  </ul>
</div>

<a id="intro"></a>
<h1>1. Introducción</h1>

<p>
En este cuaderno implementamos un <b>sistema de recomendación híbrido</b> para el subconjunto
<b>All_Beauty</b> del dataset de reseñas de Amazon. Este enfoque combina:
</p>

<ul>
  <li><b>Filtrado colaborativo Item-to-Item</b> (Notebook 1): explota patrones de co-interacción entre usuarios e ítems.</li>
  <li><b>Recomendación basada en contenido</b> (Notebook 2): explota similitud textual/semántica entre productos usando metadatos.</li>
</ul>

<p>
La motivación del enfoque híbrido es reducir limitaciones típicas de cada estrategia:
</p>

<ul>
  <li><b>Item-to-Item</b> suele sufrir en escenarios de <b>alta esparsidad</b> y <b>cold-start de ítems</b>.</li>
  <li><b>Content-Based</b> puede generar recomendaciones <b>poco diversas</b> o “encerradas” en el perfil textual.</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el recomendador híbrido busca aprovechar la fortaleza del patrón colaborativo
  cuando existe señal suficiente, y complementarlo con contenido para mejorar cobertura, robustez y cold-start.
</div>

<a id="fuentes"></a>
<h2>1.1. Fuentes de datos y estructura</h2>

<p>
Utilizamos los mismos dos ficheros ya empleados en los cuadernos previos:
</p>

<ul>
  <li><b>Dataset A — User Reviews</b>: interacciones usuario–producto (reseñas) en formato JSONL.</li>
  <li><b>Dataset B — Item Metadata</b>: metadatos del producto (título, descripción, categorías, etc.) en formato JSONL.</li>
</ul>

<p>
Campos principales (resumen):
</p>

<ul>
  <li><b>User Reviews</b>: <code>user_id</code>, <code>parent_asin</code>, <code>rating</code>, <code>timestamp</code>…</li>
  <li><b>Item Metadata</b>: <code>parent_asin</code>, <code>title</code>, <code>description</code>, <code>features</code>, <code>categories</code>…</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el sistema híbrido requiere <b>señal colaborativa</b> (interacciones) y
  <b>representación de contenido</b> (documento textual por ítem), por lo que ambos datasets son imprescindibles.
</div>

<a id="carga"></a>
<h2>1.2. Carga de datos</h2>

<p>
La carga de datos replica la lógica usada en los Notebooks 1 y 2. Asumimos que los JSONL
están accesibles desde Google Drive (o localmente en Colab). Por lo tanto, se mantiene consistencia con los cuadernos previos para garantizar
  reproducibilidad y comparación posterior (full vs muestra).
</p>

</div>

In [1]:
# Montamos nuestro Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# Importamos las librerías necesarias:
import json
import pandas as pd

# Definimos las rutas donde están ambos archivos en formato JSONL subidos a Google Colab:
path_reviews = '/content/drive/My Drive/Colab Notebooks/All_Beauty.jsonl'
path_meta    = '/content/drive/My Drive/Colab Notebooks/meta_All_Beauty.jsonl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Cargamos el dataset de reseñas (User Reviews):
reviews = []
with open(path_reviews, "r", encoding="utf-8") as f:
    for line in f:
        reviews.append(json.loads(line.strip()))
df_reviews = pd.DataFrame(reviews)

# Mostramos las primeras líneas del dataset de reseña:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [3]:
# Cargamos el dataset de metadatos (Item Metadata):
meta = []
with open(path_meta, "r", encoding="utf-8") as f:
    for line in f:
        meta.append(json.loads(line.strip()))
df_meta = pd.DataFrame(meta)

# Mostramos las primeras líneas del dataset de metadatos:
df_meta.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,None


In [4]:
# Mostramos las dimensiones ambos datasets referidos:
print("Las dimensiones de cada dataset son las siguientes:")
print("df_reviews shape:", df_reviews.shape)
print("df_meta shape:", df_meta.shape)

Las dimensiones de cada dataset son las siguientes:
df_reviews shape: (701528, 10)
df_meta shape: (112590, 14)


<a id="contexto"></a>
<h2>1.3. Contexto de los datos y definición del objetivo</h2>

<p>
El subconjunto <b>All_Beauty</b> contiene interacciones usuario–producto a partir de reseñas de Amazon.
Cada registro de reseña representa una evidencia explícita (p.ej. <code>rating</code>) y un evento temporal
(p.ej. <code>timestamp</code>), mientras que el dataset de metadatos aporta información descriptiva del producto
(p.ej. <code>title</code>, <code>description</code>, <code>features</code>, <code>categories</code>).
</p>

<p>
En este TFM, el objetivo es construir un recomendador que, dado un usuario, sea capaz de sugerir un top-N de
productos relevantes del dominio <b>belleza</b>. Para ello, se consideran dos fuentes de señal:
</p>

<ul>
  <li>
    <b>Señal colaborativa</b>: patrones de consumo/reseña compartidos entre usuarios y productos (Notebook 1).
  </li>
  <li>
    <b>Señal de contenido</b>: similitud semántica/textual entre productos usando metadatos (Notebook 2).
  </li>
</ul>

<p>
<b>Decisiones de modelado (alineadas con los cuadernos previos):</b>
</p>

<ul>
  <li>
    Se transforma la señal explícita (<code>rating</code>) a una señal implícita cuando sea necesario
    (por ejemplo, considerar interacción positiva si <code>rating ≥ 4</code>).
  </li>
  <li>
    La unidad de ítem será <code>parent_asin</code> para permitir el cruce consistente entre reseñas y metadatos.
  </li>
  <li>
    Se aplican filtros de consistencia (p.ej. eliminar registros sin <code>user_id</code> o <code>parent_asin</code>,
    y manejar valores nulos en campos textuales del contenido).
  </li>
</ul>

<p>
Finalmente, el <b>sistema híbrido</b> combinará las predicciones de ambos enfoques a nivel de score, buscando
mejorar:
</p>

<ul>
  <li><b>Calidad</b>: precisión y ranking (Precision@K, Recall@K, NDCG/MAP según se use).</li>
  <li><b>Cobertura</b>: proporción de catálogo que puede recomendarse.</li>
  <li><b>Robustez</b>: mitigación de cold-start parcial y esparsidad del colaborativo.</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el problema se plantea como recomendación Top-N en All_Beauty,
  integrando señal colaborativa (interacciones) y señal de contenido (metadatos) para mejorar calidad,
  cobertura y robustez frente a esparsidad y cold-start.
</div>

<a id="estadisticas"></a>
<h2>1.4. Estadísticas básicas del subconjunto All_Beauty</h2>

<p>
Se calculan métricas descriptivas mínimas para contextualizar el problema:
</p>

<ul>
  <li>Número de interacciones (reseñas)</li>
  <li>Número de usuarios únicos</li>
  <li>Número de ítems únicos (<code>parent_asin</code>)</li>
</ul>

<p>
(El diagnóstico profundo de esparsidad, long tail y calidad del texto se desarrolló en los Notebooks 1 y 2,
y aquí se reutiliza como referencia sin duplicarlo.)
</p>

In [5]:
# ============================================================
# Estadísticas básicas (All_Beauty)
# ============================================================

#Creamos las variables necesarias:
n_reviews = df_reviews.shape[0]
n_users   = df_reviews["user_id"].nunique()
n_items_r = df_reviews["parent_asin"].nunique()

n_items_m = df_meta["parent_asin"].nunique()
n_rows_m  = df_meta.shape[0]

# Imprimimos los resultados obtenidos:
print("\nEstadísticas básicas (All_Beauty - User Reviews):")
print(f"- Reseñas (interacciones): {n_reviews:,}".replace(",", "."))
print(f"- Usuarios únicos: {n_users:,}".replace(",", "."))
print(f"- Ítems únicos (en reseñas): {n_items_r:,}".replace(",", "."))

print("\nEstadísticas básicas (All_Beauty - Item Metadata):")
print(f"- Filas (productos): {n_rows_m:,}".replace(",", "."))
print(f"- parent_asin únicos: {n_items_m:,}".replace(",", "."))


Estadísticas básicas (All_Beauty - User Reviews):
- Reseñas (interacciones): 701.528
- Usuarios únicos: 631.986
- Ítems únicos (en reseñas): 112.565

Estadísticas básicas (All_Beauty - Item Metadata):
- Filas (productos): 112.590
- parent_asin únicos: 112.590


<hr>

<a id="modelo"></a>
<h1>2. Enfoque del modelo: Sistema de Recomendación Híbrido</h1>

<p>
En este capítulo se describe el enfoque conceptual del <b>sistema de recomendación híbrido</b> propuesto en este TFM.
Este sistema combina dos estrategias ya desarrolladas previamente:
</p>

<ul>
  <li><b>Filtrado Colaborativo Item-to-Item</b></li>
  <li><b>Recomendación Basada en Contenido</b></li>
</ul>

<p>
La motivación principal del enfoque híbrido es aprovechar la <b>complementariedad</b> entre ambos modelos,
integrando información procedente del comportamiento colectivo de los usuarios y de las características
intrínsecas de los productos.
</p>

<a id="intuicion"></a>
<h2>2.1. Intuición del enfoque híbrido</h2>

<p>
La intuición del sistema híbrido parte de una observación sencilla:
<b>los productos pueden ser relevantes para un usuario por distintos motivos</b>.
</p>

<ul>
  <li>
    Dos productos pueden ser relevantes porque <b>otros usuarios con patrones similares los han consumido conjuntamente</b>
    (señal colaborativa).
  </li>
  <li>
    Dos productos pueden ser relevantes porque <b>comparten características semánticas o funcionales similares</b>
    (señal de contenido).
  </li>
</ul>

<p>
El sistema híbrido integra ambas perspectivas para generar recomendaciones más robustas y equilibradas.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Regla práctica:</b> “recomendamos productos que son similares a los ya consumidos por el usuario,
  tanto por patrones de comportamiento global como por similitud de contenido”.
</div>

<a id="representacion"></a>
<h2>2.2. Representación de los datos</h2>

<p>
El sistema híbrido utiliza una representación dual de los datos:
</p>

<ul>
  <li>
    <b>Matriz usuario–ítem</b> para modelar interacciones y calcular similitud Item-to-Item.
  </li>
  <li>
    <b>Representación vectorial de ítems</b> basada en contenido textual (TF-IDF).
  </li>
</ul>

<p>
En ambos casos, los ítems están identificados de forma consistente mediante el campo
<code>parent_asin</code>, lo que permite integrar ambas fuentes de información.
</p>

<p>
La señal de preferencia del usuario puede definirse como:
</p>

<ul>
  <li>
    <b>Explícita:</b> utilizando directamente el <code>rating</code>.
  </li>
  <li>
    <b>Implícita:</b> transformando el rating en una interacción positiva (por ejemplo, <code>rating ≥ 4</code>).
  </li>
</ul>

<a id="senales"></a>
<h2>2.3. Señales utilizadas en el sistema híbrido</h2>

<p>
El sistema híbrido integra dos tipos de señal complementarias:
</p>

<h4>Señal colaborativa</h4>
<ul>
  <li>Interacciones usuario–producto extraídas de las reseñas.</li>
  <li>Co-ocurrencias de productos en historiales de usuarios.</li>
  <li>Similitud Item-to-Item basada en patrones de consumo compartidos.</li>
</ul>

<h4>Señal basada en contenido</h4>
<ul>
  <li>Texto descriptivo de los productos (título, descripción, características).</li>
  <li>Vectorización mediante TF-IDF.</li>
  <li>Similitud semántica entre productos usando similitud del coseno.</li>
</ul>

<p>
Ambas señales se calculan de forma independiente y se combinan posteriormente en el proceso de recomendación.
</p>

<a id="hibridacion"></a>
<h2>2.4. Estrategia de combinación (hibridación)</h2>

<p>
La estrategia adoptada es una <b>hibridación a nivel de score</b>.
Cada modelo genera un score de relevancia para un conjunto de ítems candidatos, y dichos scores se combinan
para obtener una puntuación final.
</p>

<p>
Formalmente, para un usuario <code>u</code> y un ítem candidato <code>i</code>:
</p>

<pre style="background:#f6f8fa;border:1px solid #d0d7de;padding:10px;border-radius:8px;">
score_híbrido(i | u) =
  α · score_colaborativo_norm(i | u) +
  (1 − α) · score_contenido_norm(i | u)
</pre>

<p>
donde el parámetro <b>α</b> controla el peso relativo de la señal colaborativa frente a la señal de contenido.
</p>

<p>
Esta estrategia permite ajustar el comportamiento del sistema en función de las características del dominio
y del nivel de información disponible.
</p>

<a id="recomendaciones"></a>
<h2>2.5. Generación de recomendaciones</h2>

<p>
El proceso de recomendación para un usuario sigue los siguientes pasos:
</p>

<ol>
  <li>
    Se identifican los ítems con los que el usuario ha interactuado positivamente.
  </li>
  <li>
    Para cada uno de esos ítems, se obtienen:
    <ul>
      <li>Ítems similares según el modelo Item-to-Item.</li>
      <li>Ítems similares según el modelo basado en contenido.</li>
    </ul>
  </li>
  <li>
    Se agregan los ítems candidatos, combinando los scores de ambos modelos.
  </li>
  <li>
    Se eliminan los ítems ya consumidos por el usuario y se genera un ranking Top-N final.
  </li>
</ol>

<p>
El resultado es una lista de recomendaciones que integra patrones de comportamiento colectivo
y similitud semántica entre productos.
</p>

<a id="justificacion"></a>
<h2>2.6. Justificación del enfoque híbrido</h2>

<p>
El enfoque híbrido es especialmente adecuado para el dataset All_Beauty debido a:
</p>

<ul>
  <li>Alta esparsidad de la matriz usuario–ítem.</li>
  <li>Existencia de productos con pocas interacciones.</li>
  <li>Disponibilidad de metadatos ricos en contenido textual.</li>
</ul>

<p>
Además, el sistema híbrido permite mejorar:
</p>

<ul>
  <li><b>Cobertura</b> del catálogo.</li>
  <li><b>Robustez</b> frente a cold-start parcial.</li>
  <li><b>Calidad</b> del ranking de recomendaciones.</li>
</ul>

<a id="limitaciones"></a>
<h2>2.7. Limitaciones del enfoque híbrido</h2>

<p>
A pesar de sus ventajas, el sistema híbrido presenta algunas limitaciones:
</p>

<ul>
  <li>
    <b>Mayor complejidad computacional</b> al combinar dos modelos.
  </li>
  <li>
    <b>Necesidad de calibración</b> del parámetro α.
  </li>
  <li>
    <b>Dependencia de la calidad del contenido textual</b> disponible.
  </li>
</ul>

<p>
En el siguiente capítulo se desarrollará el análisis exploratorio específico y la preparación de los datos
necesaria para implementar este sistema híbrido.
</p>

<hr>

<a id="cap3"></a>
<h1>3. Análisis exploratorio y diagnóstico del dataset para el sistema híbrido</h1>

<p>
En este capítulo se realiza un análisis exploratorio y diagnóstico <b>específico</b> orientado al diseño
del sistema de recomendación híbrido. No se repite el análisis exploratorio exhaustivo ya realizado en
los sistemas Item-to-Item y Basado en Contenido, sino que se revisan únicamente aquellos aspectos
críticos para garantizar la correcta integración de ambos enfoques.
</p>

<p>
El objetivo principal es validar que el dataset proporciona:
</p>

<ul>
  <li>Señal colaborativa suficiente para el filtrado Item-to-Item.</li>
  <li>Información de contenido adecuada para representar los ítems.</li>
  <li>Consistencia entre ambos conjuntos de datos.</li>
</ul>

<a id="consistencia"></a>
<h2>3.1. Consistencia entre interacciones y metadatos</h2>

<p>
Un requisito fundamental del sistema híbrido es que los ítems presentes en las interacciones
(usuario–producto) puedan enlazarse correctamente con sus metadatos de contenido.
</p>

<p>
Para ello, se verifica:
</p>

<ul>
  <li>El grado de solapamiento de <code>parent_asin</code> entre <b>df_reviews</b> y <b>df_meta</b>.</li>
  <li>La existencia de ítems con interacciones pero sin información de contenido.</li>
</ul>

In [6]:
# ============================================================
# Consistencia entre interacciones y metadatos
# ============================================================

items_reviews = set(df_reviews["parent_asin"].unique())
items_meta    = set(df_meta["parent_asin"].unique())

common_items  = items_reviews.intersection(items_meta)
only_reviews  = items_reviews - items_meta
only_meta     = items_meta - items_reviews

print(f"Ítems en reseñas: {len(items_reviews):,}".replace(",", "."))
print(f"Ítems en metadatos: {len(items_meta):,}".replace(",", "."))
print(f"Ítems comunes: {len(common_items):,}".replace(",", "."))
print(f"Ítems solo en reseñas: {len(only_reviews):,}".replace(",", "."))
print(f"Ítems solo en metadatos: {len(only_meta):,}".replace(",", "."))

Ítems en reseñas: 112.565
Ítems en metadatos: 112.590
Ítems comunes: 112.565
Ítems solo en reseñas: 0
Ítems solo en metadatos: 25


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el análisis muestra un solapamiento completo entre los ítems con
  interacciones y los metadatos disponibles: todos los productos presentes en las reseñas cuentan
  con información de contenido asociada. Adicionalmente, se identifican un pequeño número de ítems
  con metadatos pero sin interacciones, que constituyen un escenario natural de <i>cold-start de ítems</i>,
  donde el componente basado en contenido del sistema híbrido resulta especialmente relevante.
</div>

<a id="diagnostico"></a>
<h2>3.2. Diagnóstico de la señal colaborativa</h2>

<p>
El componente colaborativo del sistema híbrido se apoya en las interacciones
usuario–producto extraídas de las reseñas. Tal como se desarrolló en el Notebook 1,
estas interacciones permiten construir una matriz usuario–ítem de gran tamaño,
sobre la que se calculan relaciones de similitud Item-to-Item.
</p>

<p>
El análisis exploratorio realizado previamente pone de manifiesto varias características
clave del dataset:
</p>

<ul>
  <li>
    La matriz usuario–ítem presenta una <b>alta esparsidad</b>, con un gran número de
    usuarios que cuentan con historiales de interacción cortos.
  </li>
  <li>
    Existe una distribución <b>long-tail</b> en el número de interacciones por producto,
    donde una pequeña fracción de ítems concentra la mayor parte de las reseñas.
  </li>
  <li>
    A pesar de esta esparsidad, el volumen total de interacciones es suficiente para
    estimar <b>relaciones Item-to-Item estables</b> para una parte significativa del catálogo.
  </li>
</ul>

<p>
Estas propiedades hacen que el filtrado colaborativo Item-to-Item sea especialmente
adecuado en escenarios donde el usuario ha interactuado al menos con uno o varios
productos, ya que permite generar recomendaciones sin necesidad de construir un perfil
explícito de usuario.
</p>

<p>
No obstante, la presencia de productos con pocas o ninguna interacción limita la capacidad
del modelo colaborativo en escenarios de <i>cold-start de ítems</i>, donde no es posible
inferir relaciones fiables únicamente a partir del comportamiento histórico.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la señal colaborativa resulta adecuada para capturar
  relaciones Item-to-Item relevantes en una parte sustancial del catálogo, pero su
  naturaleza dispersa y la existencia de ítems con baja interacción justifican su
  combinación con un modelo basado en contenido dentro de un sistema híbrido.
</div>

<a id="diagnosticocontenido"></a>
<h2>3.3. Diagnóstico de la señal de contenido</h2>

<p>
El componente basado en contenido del sistema híbrido depende de la disponibilidad
y calidad de la información descriptiva asociada a cada producto. En el Notebook 2
se analizó la <b>cobertura real de contenido</b>, entendida como la proporción de productos
que disponen de información no vacía y semánticamente utilizable.
</p>

<p>
El análisis de los campos textuales muestra que:
</p>

<ul>
  <li>
    El campo <code>title</code> presenta una cobertura prácticamente completa y constituye
    la principal fuente de información semántica textual del catálogo.
  </li>
  <li>
    Los campos <code>description</code> y <code>features</code> presentan una cobertura
    considerablemente menor, por lo que su contribución semántica es parcial y no homogénea
    en el conjunto de productos.
  </li>
</ul>

<p>
De forma complementaria, el análisis de atributos estructurados (no textuales) revela que
campos como <code>main_category</code>, <code>details</code> y <code>store</code> presentan
una cobertura elevada, lo que permite incorporar información contextual adicional a la
representación de los ítems.
</p>

<p>
En el contexto del sistema híbrido, la señal de contenido desempeña un doble rol fundamental:
</p>

<ul>
  <li>
    Complementar la señal colaborativa en escenarios de alta esparsidad o baja interacción.
  </li>
  <li>
    Permitir la recomendación de productos nuevos o poco populares, donde el filtrado
    colaborativo no dispone de información suficiente (<i>cold-start de ítems</i>).
  </li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la disponibilidad de información de contenido, especialmente
  a través del campo <code>title</code> y de determinados atributos estructurados con alta
  cobertura, hace viable y justificada la incorporación de un modelo basado en contenido
  como componente del sistema de recomendación híbrido.
</div>

<a id="implicaciones"></a>
<h2>3.4. Implicaciones del diagnóstico para el sistema de recomendación híbrido</h2>

<p>
El análisis realizado en los apartados anteriores permite extraer una serie de implicaciones
directas para el diseño y la implementación del sistema de recomendación híbrido.
Estas implicaciones derivan tanto de las características de la señal colaborativa como
de la disponibilidad y calidad de la señal de contenido.
</p>

<p>
En primer lugar, el diagnóstico de la señal colaborativa muestra que, aunque el volumen
global de interacciones es suficiente para aprender relaciones Item-to-Item relevantes,
la alta esparsidad de la matriz usuario–ítem y la presencia de productos con pocas
interacciones limitan la capacidad del filtrado colaborativo en determinados escenarios.
</p>

<p>
En segundo lugar, el análisis de la señal de contenido evidencia que existe información
descriptiva suficiente para representar semánticamente los productos, si bien dicha
información no es homogénea en todos los campos. En particular, el campo <code>title</code>
y determinados atributos estructurados presentan una cobertura elevada, mientras que
otros campos textuales aportan información solo de forma parcial.
</p>

<p>
A partir de estos resultados, se derivan las siguientes decisiones de diseño para el
sistema híbrido:
</p>

<ul>
  <li>
    Combinar la señal colaborativa y la señal de contenido a nivel de score, de forma que
    ambas contribuyan de manera complementaria a la recomendación final.
  </li>
  <li>
    Priorizar los campos de contenido con alta cobertura y relevancia semántica en la
    representación de los ítems.
  </li>
  <li>
    Incorporar mecanismos de normalización y ponderación que permitan equilibrar la
    contribución de cada componente del sistema.
  </li>
  <li>
    Diseñar el sistema de forma que pueda generar recomendaciones tanto para usuarios
    con historial previo como para escenarios de baja interacción o <i>cold-start</i> de ítems.
  </li>
</ul>

<p>
Estas decisiones orientan directamente la fase de preparación de datos y construcción
del modelo híbrido, que se desarrollan en el siguiente capítulo.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el diagnóstico confirma que ninguna de las señales disponibles
  es suficiente por sí sola para cubrir todos los escenarios del dominio All_Beauty. La
  combinación de señal colaborativa y de contenido emerge como una solución justificada
  y necesaria, y define los requisitos técnicos que guían la preparación de datos y la
  implementación del sistema de recomendación híbrido.
</div>

<hr>

<a id="cap4"></a>
<h1>4. Preparación de datos para el sistema de recomendación híbrido</h1>

<p>
En este capítulo se describen los pasos de preparación de datos necesarios para la
construcción del sistema de recomendación híbrido. El objetivo es definir un pipeline
coherente que permita integrar de forma consistente el componente colaborativo
(Item-to-Item) y el componente basado en contenido.
</p>

<p>
La preparación de datos sigue los mismos principios metodológicos adoptados en los
Notebooks 1 y 2, garantizando la coherencia entre modelos y permitiendo comparaciones
justas en fases posteriores de evaluación.
</p>

<p>
Los pasos descritos a continuación se aplican inicialmente sobre el dataset completo.
Más adelante, se reutilizará este mismo pipeline sobre una muestra del dataset para
analizar el impacto del tamaño de los datos en el rendimiento del sistema híbrido.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> este capítulo establece un pipeline de preparación común
  que asegura la coherencia metodológica entre los distintos sistemas de recomendación
  desarrollados en este trabajo.
</div>

<a id="definicion"></a>
<h2>4.1. Definición de la señal de interacción</h2>

<p>
El sistema de recomendación híbrido se basa en las interacciones usuario–producto
extraídas de las reseñas. Aunque el dataset proporciona valoraciones explícitas
(<code>rating</code>), en este trabajo se adopta un enfoque de <b>feedback implícito</b>,
siguiendo la misma estrategia utilizada en el sistema Item-to-Item.
</p>

<p>
En concreto, se considera que una interacción es positiva cuando la valoración del
usuario es igual o superior a un umbral predefinido:
</p>

<ul>
  <li><b>Interacción positiva:</b> <code>rating ≥ 4</code></li>
</ul>

<p>
Este criterio permite simplificar la representación de las interacciones y resulta
adecuado para sistemas de recomendación orientados a ranking Top-N. Además, es
compatible tanto con el filtrado colaborativo como con el enfoque basado en contenido,
que pueden operar a partir de una única interacción positiva.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la transformación de la señal explícita en feedback
  implícito permite unificar el tratamiento de las interacciones y mantener coherencia
  con los sistemas de recomendación desarrollados previamente.
</div>

In [7]:
# ============================================================
# 4.1 Definición de la señal implícita
# ============================================================

df_interactions = df_reviews.copy()

RATING_THRESHOLD = 4  # interacción positiva si rating >= 4

df_interactions["interaction"] = (
    df_interactions["rating"] >= RATING_THRESHOLD
).astype(int)

# Nos quedamos solo con interacciones positivas
df_interactions = df_interactions[df_interactions["interaction"] == 1]

df_interactions = df_interactions[["user_id", "parent_asin", "interaction"]]

print("Interacciones positivas:", df_interactions.shape[0])
print("Usuarios únicos:", df_interactions["user_id"].nunique())
print("Ítems únicos:", df_interactions["parent_asin"].nunique())

display(df_interactions.head())

Interacciones positivas: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


,user_id,parent_asin,interaction
0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B00YQ6X8EO,1
1,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B081TJ8YS3,1
2,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,B097R46CSY,1
4,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B08BZ63GMJ,1
5,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,B00R8DXL44,1


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Aclaración metodológica:</b> las métricas obtenidas en este apartado (4.1) describen la señal implícita
  antes de construir la matriz usuario–ítem y antes de aplicar restricciones del componente colaborativo
  (p. ej., soporte mínimo por ítem). Por ello, los recuentos pueden diferir de los reportados en el Notebook 1
  en secciones posteriores donde ya se trabaja sobre la matriz CSR y se filtran ítems con baja evidencia
  colaborativa (por ejemplo, <code>nnz ≥ 2</code>).
</div>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> la transformación de la señal explícita en feedback implícito
mediante el umbral <code>rating ≥ 4</code> permite unificar el tratamiento de las interacciones
y mantener coherencia con el sistema Item-to-Item desarrollado previamente. Los resultados
obtenidos muestran que este criterio conserva un volumen elevado de interacciones positivas,
usuarios e ítems, proporcionando una base suficientemente rica para el aprendizaje de
patrones de recomendación orientados a ranking Top-N y adecuada para la integración del
componente colaborativo y el componente basado en contenido.
</div>

<a id="filtradoconsistencia"></a>
<h2>4.2. Filtrado básico de consistencia</h2>

<p>
Antes de aplicar cualquier filtrado por frecuencia o construir representaciones para
el sistema híbrido, se realiza un filtrado básico de consistencia. Este paso garantiza
la integridad estructural de los datos y evita errores posteriores en la construcción
de matrices y en el cálculo de similitudes.
</p>

<p>
En concreto, se aplican los siguientes criterios:
</p>

<ul>
  <li>Eliminación de registros sin identificador de usuario o de producto.</li>
  <li>Conservación únicamente de ítems para los que existe información de metadatos.</li>
</ul>

<p>
Este filtrado es conservador y no pretende reducir el volumen de datos, sino asegurar
la coherencia entre las distintas fuentes utilizadas por el sistema híbrido.
</p>

In [8]:
# ============================================================
# 4.2 Filtrado básico de consistencia
# ============================================================

# Eliminar registros incompletos
df_interactions = df_interactions.dropna(subset=["user_id", "parent_asin"])

# Mantener solo ítems con metadatos (coherente con Notebook 2)
valid_items = set(df_meta["parent_asin"].unique())
df_interactions = df_interactions[
    df_interactions["parent_asin"].isin(valid_items)
]

print("Tras filtrado de consistencia:")
print("Interacciones:", df_interactions.shape[0])
print("Usuarios únicos:", df_interactions["user_id"].nunique())
print("Ítems únicos:", df_interactions["parent_asin"].nunique())

Tras filtrado de consistencia:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> el filtrado básico de consistencia confirma la alta calidad
estructural del dataset, ya que no se produce pérdida de interacciones relevantes tras la
eliminación de registros incompletos y la restricción a ítems con metadatos disponibles.
Este resultado garantiza la coherencia entre las interacciones de usuario y la información
de contenido, proporcionando una base sólida y alineada para la construcción de las
representaciones necesarias en el sistema de recomendación híbrido.
</div>

<a id="filtradofrecuencia"></a>
<h2>4.3. Filtrado por frecuencia para estabilidad del sistema híbrido</h2>

<p>
Tras garantizar la consistencia estructural de los datos, se aplica un filtrado por
frecuencia de interacción con el objetivo de mejorar la estabilidad del componente
colaborativo, sin comprometer la cobertura necesaria para el componente basado en
contenido.
</p>

<p>
Siguiendo el enfoque adoptado en el sistema Item-to-Item, este filtrado se diseña de
forma <b>mínima y configurable</b>. En particular, se evita la eliminación agresiva de
usuarios o ítems, ya que el sistema híbrido debe ser capaz de operar en escenarios de
historial corto y baja evidencia colaborativa.
</p>

<p>
El filtrado se aplica de manera independiente a usuarios e ítems, permitiendo analizar
su impacto y ajustar los umbrales en función del comportamiento del modelo.
</p>

<h3>4.3.1. Filtrado mínimo de usuarios</h3>

<p>
En este subapartado se filtran los usuarios en función del número de interacciones
positivas registradas. Siguiendo la misma estrategia que en el sistema Item-to-Item,
se establece un umbral bajo que permite conservar usuarios con historiales muy cortos.
</p>

<p>
Esta decisión es deliberada: tanto el filtrado colaborativo Item-to-Item como el enfoque
basado en contenido pueden generar recomendaciones a partir de una única interacción
positiva. Eliminar estos usuarios reduciría artificialmente la cobertura del sistema
híbrido y limitaría el análisis de escenarios realistas.
</p>

In [9]:
# ============================================================
# Filtrado mínimo de usuarios
# ============================================================

MIN_USER_INTERACTIONS = 1  # configurable (p.ej. 2 para pruebas)

tmp = df_interactions.copy()

user_freq = tmp.groupby("user_id")["parent_asin"].size()
valid_users = user_freq[user_freq >= MIN_USER_INTERACTIONS].index

tmp = tmp[tmp["user_id"].isin(valid_users)].copy()

# Imprimimos los resultados obtenidos:
print("Tras filtrado de usuarios:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Tras filtrado de usuarios:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del subapartado:</b> el filtrado mínimo de usuarios con un umbral de una
interacción positiva no produce reducción en el número de usuarios ni de interacciones,
confirmando que el conjunto de datos ya cumple este criterio de forma natural. Este
resultado valida la decisión de emplear un filtrado conservador, coherente con el sistema
Item-to-Item, y permite preservar escenarios de historiales cortos en los que el
componente basado en contenido puede aportar valor adicional dentro del sistema híbrido.
</div>

<h3>4.3.2. Filtrado mínimo de ítems</h3>

<p>
En este subapartado se aplica un filtrado mínimo de ítems en función del número de
interacciones positivas registradas. Este paso tiene como objetivo reducir posibles
fuentes de ruido en el componente colaborativo, evitando ítems con evidencia
extremadamente baja para el cálculo de co-ocurrencias.
</p>

<p>
Siguiendo la misma estrategia utilizada en el sistema Item-to-Item, el umbral se
establece inicialmente en un valor bajo y configurable. De este modo, se evita eliminar
ítems potencialmente relevantes desde el punto de vista del contenido, que podrían ser
recomendados por el componente basado en contenido incluso cuando su soporte
colaborativo es limitado.
</p>

<p>
Este filtrado se concibe como un compromiso entre estabilidad colaborativa y cobertura
del catálogo, manteniendo la flexibilidad necesaria para ajustar el umbral en fases
experimentales posteriores.
</p>

In [10]:
# ============================================================
# 4.3.2 Filtrado mínimo de ítems
# ============================================================

MIN_ITEM_INTERACTIONS = 1  # configurable (p.ej. 2 o 3 en pruebas)

item_freq = tmp.groupby("parent_asin")["user_id"].size()
valid_items = item_freq[item_freq >= MIN_ITEM_INTERACTIONS].index

tmp = tmp[tmp["parent_asin"].isin(valid_items)].copy()

# Imprimimos los resultados obtenidos:
print("Tras filtrado de ítems:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Tras filtrado de ítems:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del subapartado:</b> el filtrado mínimo de ítems con un umbral de una
interacción positiva no produce reducción en el número de ítems ni de interacciones,
lo que indica que el dataset ya satisface este criterio de forma natural. Este resultado
confirma la idoneidad de un filtrado conservador en el sistema híbrido, preservando la
cobertura del catálogo y permitiendo que ítems con bajo soporte colaborativo puedan ser
recomendados a través del componente basado en contenido.
</div>

<h3>4.3.3. Dataset resultante tras el filtrado por frecuencia</h3>

<p>
El dataset resultante tras aplicar el filtrado mínimo de usuarios e ítems mantiene el
volumen original de interacciones positivas, usuarios e ítems. Este resultado confirma
que los criterios definidos son conservadores y que el conjunto de datos ya cumple de
forma natural los requisitos mínimos de frecuencia.
</p>

<p>
A partir de este punto, el pipeline continúa con el indexado de usuarios e ítems y la
construcción de las estructuras necesarias para el cálculo de similitudes y la generación
de recomendaciones.
</p>

<a id="indexado"></a>
<h2>4.4. Indexado de usuarios e ítems y construcción de la matriz CSR</h2>

<p>
En este apartado se construye la representación matricial que servirá como base para el
componente colaborativo del sistema de recomendación híbrido. En concreto, se transforma
el conjunto de interacciones filtradas en una matriz usuario–ítem dispersa (CSR),
siguiendo el mismo enfoque empleado en el sistema de recomendación Item-to-Item.
</p>

<p>
Este paso es fundamental porque permite:
</p>

<ul>
  <li>Representar de forma eficiente un dataset altamente disperso.</li>
  <li>Garantizar compatibilidad directa con el cálculo de similitudes colaborativas.</li>
  <li>Establecer una estructura común sobre la que integrar posteriormente la señal de contenido.</li>
</ul>

In [11]:
# ============================================================
# Preparación del dataset base
# ============================================================

# Importamos las librerías necesarias:
from scipy.sparse import csr_matrix
import numpy as np

tmp = df_interactions.copy()

# Imprimimos los resultados obtenidos:
print("Dataset base para la matriz:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Dataset base para la matriz:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<h3>4.4.2. Indexado de usuarios e ítems</h3>

<p>
Se construyen dos diccionarios de mapeo que asignan un índice entero a cada identificador
real:
</p>

<ul>
  <li><code>user_to_idx</code>: mapea cada usuario a un índice entero.</li>
  <li><code>item_to_idx</code>: mapea cada ítem a un índice entero.</li>
</ul>

<p>
Este mapeo permite construir la matriz CSR y, posteriormente, traducir las
recomendaciones generadas al espacio original de identificadores.
</p>

In [12]:
# ============================================================
# Indexado (ID -> índice)
# ============================================================

user_ids = tmp["user_id"].astype(str).unique()
item_ids = tmp["parent_asin"].astype(str).unique()

user_to_idx = {u: i for i, u in enumerate(user_ids)}
item_to_idx = {it: j for j, it in enumerate(item_ids)}

tmp["user_idx"] = tmp["user_id"].map(user_to_idx)
tmp["item_idx"] = tmp["parent_asin"].map(item_to_idx)

n_users = len(user_ids)
n_items = len(item_ids)

print("Usuarios indexados:", n_users)
print("Ítems indexados:", n_items)

Usuarios indexados: 455586
Ítems indexados: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> el proceso de indexado de usuarios e ítems define un espacio común de representación que permite traducir de forma eficiente los identificadores reales a índices enteros, requisito indispensable para la construcción de la matriz dispersa CSR. Este paso garantiza la coherencia entre las distintas fuentes de información y proporciona una base estable sobre la que pueden operar de manera integrada el componente colaborativo y el componente basado en contenido del sistema de recomendación híbrido.
</div>

<h3>4.4.3. Construcción de la matriz usuario–ítem (CSR)</h3>

<p>
Una vez indexados usuarios e ítems, se construye la matriz dispersa <b>usuario–ítem</b>
en formato <b>CSR (Compressed Sparse Row)</b>. Esta representación es especialmente adecuada
para datasets de recomendación, donde la mayoría de combinaciones usuario–ítem no presentan
interacción (matriz altamente dispersa).
</p>

<p>
En esta matriz:
</p>

<ul>
  <li><b>Filas:</b> usuarios (<code>user_idx</code>)</li>
  <li><b>Columnas:</b> ítems (<code>item_idx</code>)</li>
  <li><b>Valores:</b> señal implícita binaria (<code>interaction = 1</code> si existe interacción positiva)</li>
</ul>

<p>
La matriz CSR permite:
</p>

<ul>
  <li>Reducir drásticamente el uso de memoria frente a una matriz densa.</li>
  <li>Acelerar operaciones posteriores de álgebra lineal y cálculo de similitudes.</li>
  <li>Escalar el sistema colaborativo a catálogos grandes y datasets dispersos.</li>
</ul>

In [13]:
# ============================================================
# Construcción de la matriz CSR (usuario x ítem)
# ============================================================

# Valores de la matriz (señal implícita binaria):
data = tmp["interaction"].astype(np.float32).values

# Índices de filas y columnas:
rows = tmp["user_idx"].astype(np.int32).values
cols = tmp["item_idx"].astype(np.int32).values

# Dimensiones:
n_users = tmp["user_idx"].nunique()
n_items = tmp["item_idx"].nunique()

# Construcción CSR:
R = csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

print("Matriz CSR construida:")
print(" - shape (n_users, n_items):", R.shape)
print(" - nnz (no ceros / interacciones):", R.nnz)
print(" - densidad aproximada:", R.nnz / (R.shape[0] * R.shape[1]))

# Chequeo rápido de coherencia:
print("\nChequeos de consistencia:")
print(" - ¿Hay valores distintos de 1?:", np.any(R.data != 1.0))
print(" - min(data):", float(R.data.min()) if R.nnz > 0 else None)
print(" - max(data):", float(R.data.max()) if R.nnz > 0 else None)

Matriz CSR construida:
 - shape (n_users, n_items): (455586, 91187)
 - nnz (no ceros / interacciones): 494780
 - densidad aproximada: 1.1909919839927541e-05

Chequeos de consistencia:
 - ¿Hay valores distintos de 1?: True
 - min(data): 1.0
 - max(data): 10.0


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> la matriz usuario–ítem en formato CSR se ha construido correctamente a partir de las interacciones positivas indexadas, dando lugar a una representación altamente dispersa y escalable. Esta estructura permite almacenar de forma eficiente el gran volumen de usuarios e ítems del dataset y constituye la base sobre la que se implementará el componente colaborativo del sistema híbrido, manteniendo al mismo tiempo la flexibilidad necesaria para su integración con el componente basado en contenido.
</div>

<div style="background:#fff8e1;border-left:4px solid #f59e0b;padding:10px 12px;border-radius:6px;">
<b>Nota metodológica:</b> los valores almacenados en la matriz CSR representan la señal de interacción positiva previa a la aplicación de filtros colaborativos más estrictos. En fases posteriores, el componente Item-to-Item aplicará restricciones adicionales sobre esta matriz para garantizar estabilidad en el cálculo de similitudes, mientras que el componente basado en contenido operará sin requerir dichas restricciones.
</div>

<hr>
<a id="disenoeimplementacion"></a>
<h1>5. Diseño e implementación del sistema de recomendación híbrido</h1>

<p>
En este capítulo se describe el diseño y la implementación del sistema de recomendación híbrido,
que combina un enfoque de <b>Filtrado Colaborativo Item-to-Item</b> con un
<b>sistema de recomendación basado en contenido</b>.
</p>

<p>
Ambos componentes han sido desarrollados y analizados de forma independiente en capítulos anteriores.
El objetivo principal de este capítulo es <b>integrar ambas señales de recomendación</b> de manera coherente,
aprovechando sus fortalezas complementarias y mitigando sus limitaciones individuales.
</p>

<p>
El sistema híbrido permite:
</p>

<ul>
  <li>Generar recomendaciones incluso en escenarios de escasez de interacciones colaborativas.</li>
  <li>Mejorar la diversidad y relevancia de las recomendaciones.</li>
  <li>Reducir el impacto del problema de <i>cold-start</i>, especialmente para ítems nuevos.</li>
</ul>

<p>
En primer lugar, se define la <b>estrategia de hibridación</b> empleada, seguida de la implementación de
cada componente y de su combinación final para la generación de recomendaciones Top-N.
</p>

<a id="estrategia"></a>
<h2>5.1. Estrategia de hibridación</h2>

<p>
El sistema de recomendación híbrido se construye siguiendo una estrategia de
<b>hibridación tardía (<i>late fusion</i>)</b>.
En este enfoque, cada componente del sistema genera recomendaciones de manera independiente,
que posteriormente se combinan en una fase de agregación.
</p>

<p>
En concreto, el sistema integra:
</p>

<ul>
  <li>
    <b>Señal colaborativa:</b> basada en Filtrado Colaborativo Item-to-Item,
    que explota patrones de co-ocurrencia en las interacciones usuario–ítem.
  </li>
  <li>
    <b>Señal de contenido:</b> basada en la similitud semántica entre ítems,
    calculada a partir de sus metadatos textuales.
  </li>
</ul>

<p>
La hibridación tardía presenta varias ventajas en este contexto:
</p>

<ul>
  <li>Permite reutilizar modelos ya entrenados de forma independiente.</li>
  <li>Facilita el análisis individual del impacto de cada señal.</li>
  <li>Ofrece flexibilidad para ajustar pesos y estrategias de combinación.</li>
</ul>

<p>
Las recomendaciones finales se obtienen mediante una combinación ponderada de los rankings
producidos por cada componente, asegurando que ambos contribuyen al resultado final
de forma controlada.
</p>

In [14]:
# ============================================================
# Estrategia de hibridación (late fusion)
# ============================================================

# Pesos de cada componente (configurables):
ALPHA_COLLAB = 0.6    # peso del filtrado colaborativo
ALPHA_CONTENT = 0.4  # peso del sistema basado en contenido

# Comprobación de coherencia:
assert abs(ALPHA_COLLAB + ALPHA_CONTENT - 1.0) < 1e-6, \
    "Los pesos de la hibridación deben sumar 1."

# Mostramos resultados obtenidos:
print("Estrategia de hibridación definida:")
print(f" - Peso colaborativo (Item-to-Item): {ALPHA_COLLAB}")
print(f" - Peso contenido (Content-Based):   {ALPHA_CONTENT}")

Estrategia de hibridación definida:
 - Peso colaborativo (Item-to-Item): 0.6
 - Peso contenido (Content-Based):   0.4


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

En este apartado se ha definido formalmente la estrategia de hibridación del sistema de recomendación,
optando por un enfoque de <i>hibridación tardía (late fusion)</i>. Este enfoque permite integrar de forma
modular e interpretable las señales colaborativa y de contenido, manteniendo la independencia de ambos
componentes durante la generación de recomendaciones.
<br>

La asignación de pesos diferenciados (<code>0.6</code> para el componente colaborativo y <code>0.4</code> para el
componente basado en contenido) refleja una preferencia inicial por la señal colaborativa cuando existe
suficiente evidencia histórica, sin renunciar a la capacidad del sistema basado en contenido para aportar
valor en escenarios de baja densidad de interacciones o <i>cold-start</i>.
<br>

La validación explícita de la coherencia de los pesos garantiza la estabilidad del proceso de combinación
y facilita futuras fases experimentales en las que dichos pesos podrán ajustarse de forma controlada.
Esta estrategia establece una base sólida y flexible para la integración efectiva de ambos enfoques en
las siguientes etapas del sistema de recomendación híbrido.
</div>

<a id="implementacioncolaborativo"></a>
<h2>5.2. Implementación del componente colaborativo (Item-to-Item)</h2>

<p>
El componente colaborativo del sistema híbrido se basa en un enfoque de
<b>Filtrado Colaborativo Item-to-Item</b>, reutilizando el modelo desarrollado y validado
previamente en el <i>Notebook 1</i>. Este enfoque explota patrones de co-ocurrencia entre
ítems a partir de las interacciones usuario–ítem representadas en la matriz
<b>CSR</b>.
</p>

<p>
El objetivo de este apartado es integrar dicho modelo dentro del sistema híbrido
<b>sin modificar su lógica interna</b>, garantizando coherencia metodológica y permitiendo
analizar de forma aislada la contribución del componente colaborativo frente al
componente basado en contenido.
</p>

<p>
En concreto, este componente:
</p>

<ul>
  <li>
    Utiliza la matriz usuario–ítem construida en el <b>Capítulo 4</b>.
  </li>
  <li>
    Calcula similitudes entre ítems a partir de interacciones implícitas.
  </li>
  <li>
    Genera recomendaciones de ítems similares a aquellos previamente consumidos por el usuario.
  </li>
</ul>

<p>
Las recomendaciones producidas por este módulo se combinarán posteriormente con las generadas
por el sistema basado en contenido mediante una <b>estrategia de hibridación tardía (late fusion)</b>,
lo que permite ajustar de forma flexible la influencia relativa de cada señal en el ranking final.
</p>

In [15]:
# ============================================================
# Componente colaborativo: Item-to-Item
# ============================================================

from sklearn.metrics.pairwise import cosine_similarity

# ------------------------------------------------------------
# Construimos matriz ítem-usuario a partir de R (CSR usuario x ítem)
# ------------------------------------------------------------
R_item_user = R.T.tocsr()

print("Matriz ítem-usuario construida:")
print(" - shape (n_items, n_users):", R_item_user.shape)
print(" - nnz:", R_item_user.nnz)

Matriz ítem-usuario construida:
 - shape (n_items, n_users): (91187, 455586)
 - nnz: 494780


In [16]:
# ------------------------------------------------------------
# Cálculo de similitud entre ítems (coseno)
# ------------------------------------------------------------
item_similarity = cosine_similarity(R_item_user, dense_output=False)

print("\nMatriz de similitud ítem-ítem:")
print(" - shape:", item_similarity.shape)
print(" - nnz:", item_similarity.nnz)


Matriz de similitud ítem-ítem:
 - shape: (91187, 91187)
 - nnz: 345287


In [17]:
# ------------------------------------------------------------
# Función de recomendación Item-to-Item
# ------------------------------------------------------------
def recommend_item_to_item(
    user_idx,
    R,
    item_similarity,
    top_n=10
):
    """
    Recomienda ítems a un usuario a partir del modelo Item-to-Item.
    """
    # Ítems con los que el usuario ha interactuado
    user_interactions = R[user_idx].nonzero()[1]

    # Score acumulado por ítem
    scores = np.zeros(R.shape[1], dtype=np.float32)

    for it in user_interactions:
        scores += item_similarity[it].toarray().ravel()

    # Eliminamos ítems ya consumidos
    scores[user_interactions] = 0.0

    # Top-N
    recommended_idx = np.argsort(scores)[::-1][:top_n]
    return recommended_idx, scores[recommended_idx]

In [18]:
# ------------------------------------------------------------
# Ejemplo de prueba con un usuario aleatorio
# ------------------------------------------------------------
sample_user_idx = np.random.randint(0, R.shape[0])

rec_items_idx, rec_scores = recommend_item_to_item(
    user_idx=sample_user_idx,
    R=R,
    item_similarity=item_similarity,
    top_n=10
)

print("\nEjemplo de recomendación Item-to-Item:")
print("Usuario índice:", sample_user_idx)
print("Ítems recomendados (idx):", rec_items_idx)
print("Scores:", rec_scores)


Ejemplo de recomendación Item-to-Item:
Usuario índice: 19348
Ítems recomendados (idx): [63208 44519 44520 30387 30388 30389 30390 30391 30392 30393]
Scores: [0.33333334 0.16666667 0.12598816 0.         0.         0.
 0.         0.         0.         0.        ]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
El componente colaborativo Item-to-Item ha sido integrado correctamente dentro del sistema de recomendación híbrido, reutilizando de forma íntegra la lógica y estructura del modelo desarrollado en el Notebook 1. A partir de la matriz CSR usuario–ítem, se ha construido la representación ítem–usuario y se ha calculado una matriz de similitud ítem–ítem basada en la similitud del coseno, adecuada para datasets grandes y altamente dispersos.<br>
Los resultados obtenidos confirman que el modelo es capaz de generar recomendaciones Top-N coherentes para usuarios individuales, incluso en escenarios de historiales de interacción muy cortos, lo que refuerza su idoneidad para sistemas reales. Este módulo aporta una señal colaborativa sólida basada en patrones de co-ocurrencia globales y queda preparado para ser combinado, mediante hibridación tardía, con el componente basado en contenido en los siguientes apartados.
</div>

<a id="implementacioncontenido"></a>
<h2>5.3. Implementación del componente basado en contenido (Content-Based)</h2>

<p>
En este apartado se implementa el componente <b>basado en contenido</b> del sistema híbrido, reutilizando
la lógica desarrollada en el <i>Notebook 2</i>. El objetivo es calcular la similitud semántica entre productos
a partir de sus metadatos, generando recomendaciones incluso cuando la señal colaborativa sea escasa.
</p>

<p>
El flujo de trabajo se estructura en cinco pasos: (1) preparación del texto, (2) vectorización TF-IDF,
(3) cálculo de similitud ítem–ítem, (4) función de recomendación y (5) ejemplo de ejecución.
</p>

<h3>5.3.1. Preparación del texto de los metadatos</h3>

<p>
Para construir un modelo basado en contenido es necesario generar un “documento” textual por producto.
Siguiendo el criterio del <i>Notebook 2</i>, se combinan campos con carga semántica (principalmente
<b>title</b> y <b>description</b>) en un único texto por ítem.
</p>

<p>
Además, se realiza un tratamiento básico de valores nulos para evitar errores en la vectorización y
garantizar que todos los productos dispongan de una representación (aunque sea vacía).
</p>

In [19]:
# ============================================================
# Preparación del texto de metadatos
# ============================================================

# Copia de trabajo:
df_meta_cb = df_meta.copy()

# Relleno de nulos (robusto):
df_meta_cb["title"] = df_meta_cb["title"].fillna("")
df_meta_cb["description"] = df_meta_cb["description"].fillna("")

# Documento textual por ítem:
df_meta_cb["text"] = (
    df_meta_cb["title"].astype(str) + " " +
    df_meta_cb["description"].astype(str)
).str.strip()

print("Productos en df_meta_cb:", df_meta_cb.shape[0])
print("Ejemplo de texto combinado:")
display(df_meta_cb[["parent_asin", "title", "description", "text"]].head())

Productos en df_meta_cb: 112590
Ejemplo de texto combinado:


,parent_asin,title,description,text
0,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",[],"Howard LC0008 Leather Conditioner, 8-Ounce (4-..."
1,B076WQZGPM,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,[],Yes to Tomatoes Detoxifying Charcoal Cleanser ...
2,B000B658RI,Eye Patch Black Adult with Tie Band (6 Per Pack),[],Eye Patch Black Adult with Tie Band (6 Per Pac...
3,B088FKY3VD,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",[],"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4..."
4,B07NGFDN6G,Precision Plunger Bars for Cartridge Grips – 9...,[The Precision Plunger Bars are designed to wo...,Precision Plunger Bars for Cartridge Grips – 9...


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

La preparación del texto de los metadatos se ha realizado de forma correcta y coherente con el
enfoque definido en el <i>Notebook 2</i>. La combinación de los campos <code>title</code> y
<code>description</code> en un único documento textual por producto permite capturar información
semántica relevante para la construcción del modelo basado en contenido.<br>

El tratamiento explícito de valores nulos garantiza la robustez del proceso de vectorización,
asegurando que todos los productos dispongan de una representación textual válida y evitando errores
en fases posteriores. El resultado obtenido, con más de 112&nbsp;000 productos preparados, confirma
la adecuada cobertura del catálogo y sienta una base sólida para la vectorización TF-IDF y el cálculo
de similitudes semánticas entre ítems.<br>

Este paso es fundamental para permitir que el componente basado en contenido genere recomendaciones
incluso en escenarios de baja evidencia colaborativa, contribuyendo así a mejorar la cobertura y la
capacidad de generalización del sistema de recomendación híbrido.
</div>

<p>
Aunque en el sistema basado en contenido puro se utilizaron más campos de metadatos, en el sistema híbrido se optó deliberadamente por una representación textual más contenida, basada únicamente en el título y la descripción. Esta decisión responde al principio de complementariedad propio de los sistemas híbridos: el objetivo del componente basado en contenido no es maximizar la señal semántica de forma aislada, sino aportar información adicional y no redundante al componente colaborativo. De este modo, se prioriza una señal textual estable, interpretable y menos ruidosa, adecuada para ser combinada mediante hibridación tardía con el filtrado colaborativo Item-to-Item.
</p>

<h3>5.3.2. Vectorización TF-IDF de los ítems</h3>

<p>
Una vez construido el documento textual por producto en el apartado anterior (5.3.1), se transforma
cada ítem en una representación numérica mediante <b>TF-IDF</b> (<i>Term Frequency – Inverse Document Frequency</i>).
</p>

<p>
TF-IDF asigna un peso más alto a términos informativos: aquellos que aparecen con frecuencia en un
producto concreto pero no son comunes en todo el catálogo. Esto permite capturar mejor la semántica
discriminativa de los ítems y es especialmente adecuado como base para calcular similitudes mediante
coseno.
</p>

<p>
Para mantener eficiencia computacional en un catálogo grande, se limita el tamaño del vocabulario
con <code>max_features</code> y se incluyen <code>n-grams</code> (unigramas y bigramas) para capturar expresiones cortas
relevantes (p. ej., “hair oil”, “face mask”).
</p>

In [20]:
# ============================================================
# Vectorización TF-IDF
# ============================================================

from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizador (ajustable según memoria/tiempo):
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2   # ignora términos extremadamente raros (reduce ruido)
)

tfidf_matrix = tfidf_vectorizer.fit_transform(df_meta_cb["text"])

# Mostramos los resultados obtenidos:
print("Matriz TF-IDF construida:")
print(" - shape (n_items, n_features):", tfidf_matrix.shape)
print(" - nnz (no ceros):", tfidf_matrix.nnz)
print(" - vocab_size:", len(tfidf_vectorizer.get_feature_names_out()))

Matriz TF-IDF construida:
 - shape (n_items, n_features): (112590, 5000)
 - nnz (no ceros): 2384067
 - vocab_size: 5000


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

La vectorización TF-IDF transforma los metadatos textuales de los productos en una representación numérica
dispersa y de alta dimensionalidad, adecuada para capturar similitudes semánticas entre ítems en un
catálogo de gran tamaño. La matriz resultante presenta una dimensionalidad de
<code>(112 590 × 5 000)</code>, lo que evidencia la escalabilidad del enfoque y su viabilidad computacional
en contextos reales.

El uso de un vocabulario limitado y de n-gramas permite equilibrar expresividad semántica y eficiencia,
reduciendo el ruido asociado a términos poco informativos sin perder capacidad discriminativa.
La elevada dispersión de la matriz confirma la idoneidad de esta representación para el cálculo eficiente
de similitudes mediante coseno.

Esta representación TF-IDF constituye la base del componente basado en contenido del sistema híbrido,
proporcionando una señal semántica complementaria al filtrado colaborativo Item-to-Item y preparada para
ser integrada posteriormente mediante una estrategia de hibridación tardía.
</div>

<h3>5.3.3. Cálculo de similitud basada en contenido (Top-K, escalable)</h3>

<p>
Calcular la matriz completa de similitud ítem–ítem resulta inviable en catálogos grandes debido a su complejidad
O(<i>n</i><sup>2</sup>) en memoria y tiempo. Por ello, en el sistema híbrido se adopta una estrategia más eficiente:
en lugar de construir una matriz densa/total, se calcula para cada producto únicamente su vecindario de
<b>Top-K ítems más similares</b>.
</p>

<p>
Este enfoque es equivalente conceptualmente, pero:
</p>

<ul>
  <li><b>Reduce drásticamente memoria</b>, almacenando solo K similitudes por ítem.</li>
  <li><b>Escala</b> a catálogos grandes sin necesidad de computar todas las parejas.</li>
  <li>Es directamente utilizable para ranking y para hibridación tardía con el componente colaborativo.</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
La señal de contenido se modela mediante un vecindario Top-K por ítem, lo que permite capturar similitud semántica
de forma eficiente y compatible con el enfoque híbrido basado en ranking (late fusion), evitando el coste prohibitivo
de una matriz completa ítem–ítem.
</div>

In [21]:
# ============================================================
# Cálculo de similitud basada en contenido (Top-K vecinos)
# ============================================================

# Imprimimos la librería necesaria:
from sklearn.neighbors import NearestNeighbors

# K vecinos por item (ajustable):
TOPK_CONTENT = 50

# Modelo kNN sobre TF-IDF (métrica coseno):
nn = NearestNeighbors(
    n_neighbors=TOPK_CONTENT + 1,  # +1 porque el primer vecino suele ser el propio ítem
    metric="cosine",
    algorithm="brute",
    n_jobs=-1
)

nn.fit(tfidf_matrix)

# Obtenemos vecinos para todos los items (esto NO construye matriz NxN):
distances, indices = nn.kneighbors(tfidf_matrix, return_distance=True)

# Convertimos distancia coseno a similitud coseno: sim = 1 - dist:
sims = 1.0 - distances

# Mostramos los resultados siguientes:
print("Vecindario basado en contenido calculado:")
print(" - indices shape:", indices.shape)
print(" - sims shape:", sims.shape)

# Ejemplo: vecinos del item 0:
print("\nEjemplo vecinos item 0 (primeros 5):")
print(list(zip(indices[0][:5], sims[0][:5])))

Vecindario basado en contenido calculado:
 - indices shape: (112590, 51)
 - sims shape: (112590, 51)

Ejemplo vecinos item 0 (primeros 5):
[(np.int64(0), np.float64(1.0)), (np.int64(89502), np.float64(1.0)), (np.int64(92424), np.float64(0.6737292843378253)), (np.int64(45496), np.float64(0.592440317414264)), (np.int64(87294), np.float64(0.592440317414264))]


In [22]:
# Función para generar recomendaciones content-based con estos vecinos:
def recommend_content_based(item_idx, indices, sims, top_n=10):
    """
    Devuelve Top-N ítems similares a item_idx según contenido.
    indices/sims vienen del kneighbors sobre TF-IDF.
    """
    neigh_idx = indices[item_idx]
    neigh_sim = sims[item_idx]

    # quitamos el propio item si aparece el primero
    mask = neigh_idx != item_idx
    neigh_idx = neigh_idx[mask]
    neigh_sim = neigh_sim[mask]

    # top_n
    order = np.argsort(neigh_sim)[::-1][:top_n]
    return neigh_idx[order], neigh_sim[order]

# Ejemplo rápido:
item_example = 0
rec_idx, rec_sim = recommend_content_based(item_example, indices, sims, top_n=10)
print("Recomendaciones content-based:", rec_idx)
print("Similitudes:", rec_sim)

Recomendaciones content-based: [89502 92424 45496 87294 26553 11802 73038 84607 45038 47026]
Similitudes: [1.         0.67372928 0.59244032 0.59244032 0.57571647 0.56728876
 0.56308404 0.55446668 0.55176741 0.53915269]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
Se ha implementado y validado con éxito el cálculo de similitud basada en contenido mediante un enfoque <b>Top-K vecinos</b> sobre representaciones TF-IDF, evitando la construcción de una matriz completa ítem–ítem y garantizando la escalabilidad del sistema.<br>
La función de recomendación desarrollada permite generar listas Top-N de productos similares a un ítem dado, excluyendo correctamente el propio ítem y ordenando los candidatos por similitud coseno decreciente. El ejemplo ejecutado confirma que el modelo identifica productos semánticamente relacionados, con valores de similitud coherentes y decrecientes.<br>
Este componente proporciona una señal de contenido interpretable y eficiente, preparada para integrarse en el sistema de recomendación híbrido mediante <b>hibridación tardía</b>, complementando la señal colaborativa Item-to-Item especialmente en escenarios de baja evidencia colaborativa.
</div>

<a id="cap5_4"></a>
<h2>5.4. Combinación de recomendaciones (hibridación tardía)</h2>

<p>
Una vez implementados los dos componentes del sistema híbrido —el <b>modelo colaborativo Item-to-Item</b> (Sección 5.2)
y el <b>modelo basado en contenido</b> (Sección 5.3)—, el siguiente paso consiste en <b>fusionar sus resultados</b> para obtener
un ranking final de recomendación.
</p>

<p>
En este trabajo se adopta una estrategia de <b>hibridación tardía (late fusion)</b>, donde cada componente produce de forma
independiente una lista de ítems candidatos con sus puntuaciones, y posteriormente se realiza una <b>agregación</b> controlada
en una fase final.
</p>

<p>
Este enfoque es especialmente adecuado en nuestro caso porque:
</p>

<ul>
  <li>Permite mantener intacta la lógica interna de cada recomendador (comparabilidad y modularidad).</li>
  <li>Facilita ajustar la contribución relativa de cada señal mediante pesos (<code>ALPHA_COLLAB</code>, <code>ALPHA_CONTENT</code>).</li>
  <li>Ofrece robustez ante escenarios de <i>cold-start</i> (ítems nuevos o usuarios con pocas interacciones).</li>
</ul>

<p>
En los siguientes subapartados se define el mecanismo concreto de combinación: normalización de scores, fusión ponderada,
eliminación de ítems ya consumidos y estrategia de <i>fallback</i> cuando una señal no produce candidatos.
</p>

<h3>5.4.1. Normalización de puntuaciones y fusión ponderada</h3>

<p>
Un reto habitual al combinar recomendadores distintos es que sus <b>puntuaciones no son directamente comparables</b>.
El componente colaborativo (Item-to-Item) produce scores derivados de similitudes por co-ocurrencia, mientras que el componente
basado en contenido produce similitudes semánticas (coseno sobre TF-IDF). Aunque ambas magnitudes están relacionadas con “similitud”,
sus escalas y distribuciones pueden diferir.
</p>

<p>
Para evitar que un recomendador domine al otro únicamente por escala, aplicamos una <b>normalización por lista</b>
(<i>min-max</i> sobre el conjunto de candidatos devueltos). Tras normalizar, realizamos una <b>fusión lineal ponderada</b>:
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Score híbrido:</b>
  <code>score_final(i) = ALPHA_COLLAB · score_collab_norm(i) + ALPHA_CONTENT · score_content_norm(i)</code>
</div>

<p>
La fusión se realiza sobre la <b>unión de candidatos</b> generados por ambos componentes. Si un ítem aparece solo en uno de los dos,
se asigna score 0 en el otro componente (tras normalización) y se mantiene en el ranking final. De esta forma, el sistema híbrido
conserva la capacidad de:
</p>

<ul>
  <li><b>Explotar patrones colaborativos</b> cuando existe suficiente evidencia de interacción.</li>
  <li><b>Complementar con contenido</b> en ítems con baja interacción o en escenarios de arranque en frío.</li>
</ul>

<p>
A continuación se implementa una función de fusión genérica que combina dos listas de recomendaciones (ítems + scores) en un ranking final.
</p>

In [23]:
# ============================================================
# Normalización de puntuaciones y fusión ponderada
# ============================================================

def minmax_normalize(scores: np.ndarray) -> np.ndarray:
    """
    Normaliza un vector de scores con Min-Max a rango [0, 1].
    Si todos los valores son iguales o el vector está vacío, devuelve ceros.
    """
    scores = np.asarray(scores, dtype=np.float32)
    if scores.size == 0:
        return scores
    s_min = float(scores.min())
    s_max = float(scores.max())
    if s_max - s_min < 1e-12:
        return np.zeros_like(scores, dtype=np.float32)
    return (scores - s_min) / (s_max - s_min)

def fuse_rankings_weighted(
    items_collab, scores_collab,
    items_content, scores_content,
    alpha_collab=0.6, alpha_content=0.4,
    top_n=10
):
    """
    Fusiona dos rankings (colaborativo y contenido) mediante:
    - normalización Min-Max por lista
    - combinación ponderada (late fusion)

    Devuelve:
    - top_items: índices de ítems recomendados
    - top_scores: scores finales
    """
    assert abs(alpha_collab + alpha_content - 1.0) < 1e-6, "Los pesos deben sumar 1."

    items_collab = np.asarray(items_collab, dtype=np.int64)
    scores_collab = np.asarray(scores_collab, dtype=np.float32)

    items_content = np.asarray(items_content, dtype=np.int64)
    scores_content = np.asarray(scores_content, dtype=np.float32)

    # Normalización por lista (solo dentro del conjunto de candidatos):
    sc_norm = minmax_normalize(scores_collab)
    sb_norm = minmax_normalize(scores_content)

    # Unión de candidatos:
    all_items = np.unique(np.concatenate([items_collab, items_content]))

    # Mapear item -> score_normalizado (si no está, score = 0):
    collab_dict = {int(i): float(s) for i, s in zip(items_collab, sc_norm)}
    content_dict = {int(i): float(s) for i, s in zip(items_content, sb_norm)}

    final_scores = np.zeros(all_items.shape[0], dtype=np.float32)

    for k, it in enumerate(all_items):
        final_scores[k] = (
            alpha_collab * collab_dict.get(int(it), 0.0) +
            alpha_content * content_dict.get(int(it), 0.0)
        )

    # Orden descendente:
    order = np.argsort(final_scores)[::-1]
    all_items = all_items[order]
    final_scores = final_scores[order]

    # Top-N:
    top_items = all_items[:top_n]
    top_scores = final_scores[:top_n]

    return top_items, top_scores


# -------------------------
# Mini-test rápido (sanity check)
# -------------------------
# Simulamos dos rankings con escalas distintas para comprobar fusión:
_demo_items_collab  = np.array([10, 20, 30, 40])
_demo_scores_collab = np.array([0.2, 0.15, 0.01, 0.005])  # escala "pequeña"
_demo_items_content  = np.array([20, 50, 60])
_demo_scores_content = np.array([0.9, 0.7, 0.2])          # escala "grande"

demo_items, demo_scores = fuse_rankings_weighted(
    _demo_items_collab, _demo_scores_collab,
    _demo_items_content, _demo_scores_content,
    alpha_collab=ALPHA_COLLAB, alpha_content=ALPHA_CONTENT,
    top_n=10
)

print("Demo - ítems fusionados:", demo_items)
print("Demo - scores finales:", demo_scores)

Demo - ítems fusionados: [20 10 50 30 60 40]
Demo - scores finales: [0.84615386 0.6        0.2857143  0.01538461 0.         0.        ]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
Los resultados obtenidos confirman la correcta implementación del proceso de normalización de puntuaciones y fusión ponderada mediante hibridación tardía. La normalización Min–Max aplicada de forma independiente a cada lista de recomendaciones permite hacer comparables las puntuaciones generadas por el componente colaborativo y el componente basado en contenido, evitando que uno de ellos domine el ranking final debido únicamente a diferencias de escala.<br>
La fusión lineal ponderada sobre la unión de candidatos demuestra que ambos componentes contribuyen de forma controlada al score final, respetando los pesos definidos (<code>ALPHA_COLLAB</code> y <code>ALPHA_CONTENT</code>) y manteniendo ítems procedentes de cualquiera de las dos señales. El mini-test de validación confirma que la estrategia prioriza adecuadamente ítems bien valorados por ambas fuentes, al tiempo que preserva recomendaciones exclusivas de un solo componente cuando aportan valor.<br>
Este enfoque proporciona un mecanismo de combinación flexible, interpretable y robusto, sentando una base sólida para la generación del ranking híbrido final y permitiendo ajustar fácilmente la influencia relativa de cada señal en función de distintos escenarios experimentales.
</div>

<h3>5.4.2. Generación del ranking híbrido final</h3>

Una vez definidos los mecanismos de normalización y fusión ponderada en el apartado anterior, el siguiente paso consiste en aplicar dicha estrategia para generar el <b>ranking híbrido final por usuario</b>.

Para cada usuario, el sistema:
<ul>
  <li>Obtiene una lista de recomendaciones del componente colaborativo (Item-to-Item).</li>
  <li>Obtiene una lista de recomendaciones del componente basado en contenido.</li>
  <li>Normaliza las puntuaciones de cada lista de forma independiente.</li>
  <li>Fusiona ambas listas mediante una combinación lineal ponderada.</li>
</ul>

El resultado es un ranking único que integra patrones de co-ocurrencia y similitud semántica, permitiendo:
<ul>
  <li>Priorizar ítems respaldados por ambas señales.</li>
  <li>Incluir ítems con baja evidencia colaborativa gracias al contenido.</li>
  <li>Mantener flexibilidad en la contribución relativa de cada componente.</li>
</ul>

Este ranking híbrido constituye la salida final del sistema de recomendación propuesto y será la base para su análisis y evaluación posterior.
</div>


In [24]:
# ============================================================
# Generación del ranking híbrido final por usuario
# ============================================================

def recommend_hybrid(
    user_idx,
    R,
    item_similarity,
    content_indices,
    content_sims,
    alpha_collab=0.6,
    alpha_content=0.4,
    top_n=10
):
    """
    Genera recomendaciones híbridas para un usuario combinando:
    - Item-to-Item colaborativo
    - Content-Based (TF-IDF + kNN)
    """

    # -----------------------------
    # 1) Recomendación colaborativa
    # -----------------------------
    items_collab, scores_collab = recommend_item_to_item(
        user_idx=user_idx,
        R=R,
        item_similarity=item_similarity,
        top_n=top_n * 2  # ampliamos para mayor diversidad
    )

    # -----------------------------
    # 2) Recomendación basada en contenido
    #    (a partir de ítems consumidos)
    # -----------------------------
    user_items = R[user_idx].nonzero()[1]

    content_candidates = []
    content_scores = []

    for it in user_items:
        rec_idx, rec_sim = recommend_content_based(
            item_idx=it,
            indices=content_indices,
            sims=content_sims,
            top_n=top_n
        )
        content_candidates.extend(rec_idx)
        content_scores.extend(rec_sim)

    # Si el usuario no tiene interacciones, devolvemos solo colaborativo
    if len(content_candidates) == 0:
        return items_collab[:top_n], scores_collab[:top_n]

    # Convertimos a arrays
    items_content = np.array(content_candidates, dtype=np.int64)
    scores_content = np.array(content_scores, dtype=np.float32)

    # -----------------------------
    # 3) Fusión ponderada
    # -----------------------------
    hybrid_items, hybrid_scores = fuse_rankings_weighted(
        items_collab, scores_collab,
        items_content, scores_content,
        alpha_collab=alpha_collab,
        alpha_content=alpha_content,
        top_n=top_n
    )

    return hybrid_items, hybrid_scores

In [25]:
# ============================================================
# Ejemplo de recomendación híbrida para un usuario aleatorio
# ============================================================

sample_user_idx = np.random.randint(0, R.shape[0])

hyb_items, hyb_scores = recommend_hybrid(
    user_idx=sample_user_idx,
    R=R,
    item_similarity=item_similarity,
    content_indices=indices,
    content_sims=sims,
    alpha_collab=ALPHA_COLLAB,
    alpha_content=ALPHA_CONTENT,
    top_n=10
)

print("Usuario índice:", sample_user_idx)
print("Ítems recomendados (híbrido):", hyb_items)
print("Scores híbridos:", hyb_scores)

Usuario índice: 29903
Ítems recomendados (híbrido): [108410  22245  15411  49381  53099  85362   3712  14672  91186 100840]
Scores híbridos: [0.4       0.4       0.4       0.0749248 0.0749248 0.0749248 0.0749248
 0.0749248 0.        0.       ]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
Los resultados obtenidos confirman la correcta implementación del proceso de generación del ranking híbrido final por usuario. A partir de las recomendaciones individuales producidas por el componente colaborativo Item-to-Item y por el componente basado en contenido, el sistema es capaz de integrar ambas señales en un único ranking coherente mediante normalización y fusión ponderada.<br>
El ranking híbrido resultante prioriza aquellos ítems respaldados simultáneamente por patrones de co-ocurrencia y similitud semántica, al tiempo que incorpora ítems propuestos únicamente por uno de los componentes cuando la evidencia de la otra señal es limitada. Este comportamiento es consistente con los objetivos del sistema híbrido y refleja una combinación equilibrada entre explotación colaborativa y generalización basada en contenido.<br>
Asimismo, la parametrización mediante los pesos <code>ALPHA_COLLAB</code> y <code>ALPHA_CONTENT</code> permite ajustar de forma flexible la influencia relativa de cada componente, facilitando la adaptación del sistema a distintos escenarios, como usuarios con historiales extensos o situaciones de arranque en frío.<br>
En conjunto, este apartado valida que el sistema híbrido produce recomendaciones finales interpretables, escalables y metodológicamente alineadas con la estrategia de hibridación tardía definida, sentando una base sólida para su evaluación y análisis en capítulos posteriores.
</div>

<a id="cap5_5"></a>
<h2>5.5. Síntesis del diseño e implementación del sistema de recomendación híbrido</h2>

En este capítulo se ha diseñado e implementado un sistema de recomendación híbrido basado en una estrategia de <i>hibridación tardía (late fusion)</i>, integrando de forma coherente un componente colaborativo Item-to-Item y un componente basado en contenido construido a partir de metadatos textuales.

El proceso se ha estructurado de manera modular, reutilizando modelos previamente desarrollados y validados de forma independiente. El componente colaborativo explota patrones de co-ocurrencia entre ítems a partir de interacciones implícitas usuario–ítem, mientras que el componente basado en contenido captura similitud semántica mediante representaciones TF-IDF y vecindarios kNN sobre el espacio textual.

La estrategia de combinación se fundamenta en la normalización de puntuaciones por lista y en una fusión lineal ponderada, lo que permite integrar señales heterogéneas evitando dominancias artificiales debidas a diferencias de escala. Esta aproximación garantiza que ambos componentes contribuyan de forma controlada al ranking final, manteniendo flexibilidad para ajustar su peso relativo en función del contexto.

El sistema resultante presenta varias propiedades clave: capacidad de explotar evidencia colaborativa cuando existe suficiente historial, robustez frente a escenarios de baja interacción gracias al contenido, y escalabilidad al trabajar con representaciones dispersas y vecindarios acotados. Además, la arquitectura modular facilita su análisis, extensión y evaluación posterior.

En conjunto, este capítulo establece una base metodológica y técnica sólida para el análisis del rendimiento del sistema híbrido, que será abordado en el capítulo siguiente mediante métricas y experimentos de evaluación adecuados.
</div>

<hr>
<a id="cap6"></a>
<h1>6. Evaluación del sistema de recomendación híbrido</h1>

<p>
Tras el diseño e implementación del sistema de recomendación híbrido en los capítulos anteriores,
este capítulo tiene como objetivo evaluar empíricamente la <b>calidad de las recomendaciones generadas</b>.
La evaluación se centra en analizar hasta qué punto el sistema híbrido es capaz de producir rankings
relevantes para los usuarios, comparando su comportamiento con el de los modelos base que lo componen:
el sistema colaborativo Item-to-Item y el sistema basado en contenido.
</p>

<p>
Dado que el objetivo del sistema no es la predicción explícita de valoraciones, sino la generación de
<b>listas Top-N de ítems relevantes</b>, se emplean métricas específicas de sistemas de recomendación que
evalúan la calidad del ranking. Estas métricas permiten analizar tanto la precisión de las recomendaciones
como su capacidad de cobertura sobre el catálogo.
</p>

<p>
La evaluación se plantea en un entorno <i>offline</i>, utilizando interacciones implícitas extraídas del
dataset y un protocolo experimental coherente con la naturaleza de los datos. Los aspectos relacionados
con la eficiencia computacional y la escalabilidad del sistema se abordan de forma independiente en el
capítulo siguiente.
</p>

<a id="cap6_1"></a>
<h2>6.1. Objetivos de la evaluación</h2>

<p>
El objetivo principal de este capítulo es analizar el rendimiento del sistema de recomendación híbrido
propuesto en términos de calidad de las recomendaciones. Para ello, se definen una serie de objetivos
específicos que guían el proceso de evaluación experimental.
</p>

<ul>
  <li>
    Evaluar la capacidad del sistema híbrido para generar recomendaciones relevantes mediante métricas
    estándar de ranking Top-N.
  </li>
  <li>
    Comparar el rendimiento del sistema híbrido frente a sus componentes individuales: el sistema
    colaborativo Item-to-Item y el sistema basado en contenido.
  </li>
  <li>
    Analizar el impacto de la hibridación tardía en la calidad de las recomendaciones, identificando
    posibles mejoras en escenarios de baja evidencia colaborativa.
  </li>
  <li>
    Estudiar la cobertura del catálogo alcanzada por cada enfoque, evaluando la capacidad del sistema
    híbrido para recomendar un conjunto más diverso de ítems.
  </li>
</ul>

<p>
A través de estos objetivos, se pretende validar empíricamente que la combinación de señales colaborativas
y de contenido proporciona una mejora global respecto al uso de cada modelo de forma aislada, justificando
así el enfoque híbrido adoptado.
</p>

<a id="cap6_2"></a>
<h2>6.2. Protocolo experimental</h2>

<p>
Para evaluar de forma coherente la calidad de las recomendaciones generadas por los distintos sistemas,
se define un protocolo experimental común que se aplica tanto al sistema colaborativo Item-to-Item,
al sistema basado en contenido y al sistema híbrido propuesto.
</p>

<p>
Dado que el dataset no dispone de información temporal suficientemente detallada para realizar una
evaluación estrictamente cronológica, se adopta un esquema de evaluación <b>offline</b> basado en la
división de las interacciones de cada usuario en conjuntos de entrenamiento y prueba.
</p>

<h3>6.2.1. Definición de la señal de relevancia</h3>

<p>
Las interacciones usuario–ítem se transforman en una señal implícita de relevancia, considerando como
interacciones positivas aquellas valoraciones con <code>rating ≥ 4</code>. Esta decisión permite reducir
el ruido asociado a valoraciones neutras o negativas y es coherente con el enfoque adoptado en los
capítulos anteriores.
</p>

<p>
Las interacciones positivas se interpretan como evidencia de interés del usuario por el ítem, y se
utilizan como referencia para evaluar si las recomendaciones generadas por el sistema son relevantes.
</p>

<h3>6.2.2. División entrenamiento–prueba</h3>

<p>
Para cada usuario, se separan sus interacciones positivas en dos subconjuntos:
</p>

<ul>
  <li>
    <b>Conjunto de entrenamiento:</b> utilizado para construir los modelos de recomendación y generar
    recomendaciones.
  </li>
  <li>
    <b>Conjunto de prueba:</b> utilizado exclusivamente para evaluar la calidad de las recomendaciones
    generadas.
  </li>
</ul>

<p>
La división se realiza de forma aleatoria a nivel de usuario, asegurando que cada usuario disponga al
menos de una interacción en el conjunto de entrenamiento. Este enfoque permite simular un escenario
realista en el que el sistema dispone de un historial parcial del usuario.
</p>

<h3>6.2.3. Configuración de evaluación Top-N</h3>

<p>
La evaluación se realiza en un contexto de recomendación Top-N. Para cada usuario del conjunto de
prueba, el sistema genera una lista ordenada de <code>N</code> ítems recomendados, eliminando aquellos
productos con los que el usuario ya ha interactuado en el conjunto de entrenamiento.
</p>

<p>
En este trabajo se consideran distintos valores de <code>N</code> (por ejemplo, <code>N = 5</code> y
<code>N = 10</code>) con el objetivo de analizar el comportamiento del sistema en listas cortas y
moderadas, habituales en aplicaciones reales.
</p>

<h3>6.2.4. Modelos evaluados</h3>

<p>
El protocolo experimental se aplica de forma homogénea a los siguientes modelos:
</p>

<ul>
  <li><b>Filtrado colaborativo Item-to-Item</b> (baseline colaborativo).</li>
  <li><b>Sistema basado en contenido</b> (baseline basado en metadatos textuales).</li>
  <li><b>Sistema de recomendación híbrido</b> mediante hibridación tardía.</li>
</ul>

<p>
Esta configuración permite realizar una comparación justa entre enfoques y analizar el impacto real
de la hibridación sobre la calidad de las recomendaciones.
</p>


<a id="cap6_3"></a>
<h2>6.3. Métricas de evaluación</h2>

<p>
Dado que el objetivo del sistema de recomendación es generar listas ordenadas de ítems relevantes
(<i>Top-N recommendation</i>), la evaluación se realiza mediante métricas específicas de ranking.
Estas métricas permiten cuantificar hasta qué punto los ítems relevantes para un usuario aparecen
en las primeras posiciones de la lista recomendada.
</p>

<p>
Las métricas seleccionadas se centran en dos aspectos complementarios: la <b>precisión del ranking</b>
y la <b>capacidad de cobertura</b> del sistema. A continuación se describen las métricas utilizadas
en este trabajo.
</p>

<h3>6.3.1. Precision@K</h3>

<p>
La métrica <b>Precision@K</b> mide la proporción de ítems relevantes entre los <code>K</code> primeros
elementos recomendados al usuario.
</p>

<p>
Formalmente, se define como:
</p>

<p style="margin-left:20px;">
<code>Precision@K = (número de ítems relevantes en el Top-K) / K</code>
</p>

<p>
Esta métrica penaliza la inclusión de ítems no relevantes en las primeras posiciones del ranking
y es especialmente adecuada cuando el usuario solo presta atención a un número reducido de
recomendaciones.
</p>

<h3>6.3.2. Recall@K</h3>

<p>
La métrica <b>Recall@K</b> mide la proporción de ítems relevantes del conjunto de prueba que aparecen
en el Top-K recomendado.
</p>

<p style="margin-left:20px;">
<code>Recall@K = (número de ítems relevantes en el Top-K) / (número total de ítems relevantes)</code>
</p>

<p>
A diferencia de la precisión, el recall evalúa la capacidad del sistema para recuperar la mayor
cantidad posible de ítems relevantes, siendo especialmente informativo en escenarios donde los
usuarios disponen de múltiples ítems relevantes.
</p>

<h3>6.3.3. NDCG@K</h3>

<p>
La métrica <b>Normalized Discounted Cumulative Gain (NDCG@K)</b> evalúa la calidad del ranking teniendo
en cuenta no solo qué ítems relevantes aparecen, sino también <b>su posición</b> dentro de la lista.
</p>

<p>
Los ítems relevantes situados en posiciones más altas contribuyen más al valor de la métrica,
siguiendo un factor de descuento logarítmico.
</p>

<p>
NDCG está normalizada en el rango <code>[0, 1]</code>, lo que permite comparar resultados entre usuarios
con distinto número de ítems relevantes.
</p>

<h3>6.3.4. Cobertura del catálogo</h3>

<p>
La <b>cobertura</b> mide la proporción de ítems distintos del catálogo que aparecen al menos una vez
en las listas de recomendaciones generadas para el conjunto de usuarios evaluados.
</p>

<p style="margin-left:20px;">
<code>Coverage = (número de ítems recomendados) / (número total de ítems del catálogo)</code>
</p>

<p>
Esta métrica permite analizar la capacidad del sistema para recomendar un conjunto diverso de
productos, evitando concentrarse únicamente en los ítems más populares. En sistemas híbridos,
una mejora en cobertura suele ser indicativa de una mejor generalización del modelo.
</p>

<p>
En conjunto, estas métricas proporcionan una visión equilibrada del rendimiento del sistema,
permitiendo evaluar tanto la relevancia de las recomendaciones como su alcance sobre el catálogo.
</p>

<a id="cap6_4"></a>
<h2>6.4. Configuración experimental y resultados</h2>

<p>
En este apartado se presenta la configuración experimental utilizada para evaluar de forma comparativa
los tres sistemas de recomendación desarrollados en este trabajo: el modelo colaborativo Item-to-Item,
el modelo basado en contenido y el sistema híbrido propuesto.
</p>

<p>
La evaluación se realiza en un entorno <b>offline</b> siguiendo el protocolo descrito en las secciones
anteriores (evaluación Top-N sobre interacciones positivas). Sin embargo, debido a la alta dispersión
del dataset y la existencia de usuarios con historiales extremadamente cortos, se ajusta la estrategia
de partición entrenamiento–prueba para obtener métricas más estables e interpretables.
</p>

<p>
En concreto, se introducen dos decisiones metodológicas clave:
</p>

<ul>
  <li>
    <b>Filtrado de usuarios con historial mínimo</b>, con el objetivo de garantizar que el sistema dispone
    de suficiente información para generar recomendaciones significativas.
  </li>
  <li>
    <b>Partición Leave-One-Out (LOO)</b>, ampliamente utilizada en la literatura de recomendación, que consiste
    en reservar una interacción positiva como conjunto de prueba y utilizar el resto como entrenamiento.
  </li>
</ul>

<p>
A partir de esta configuración, se calculan las métricas Top-N definidas previamente y se comparan
los resultados obtenidos por cada modelo, analizando el impacto real de la hibridación.
</p>

<a id="cap6_4_1"></a>
<h3>6.4.1. Selección de usuarios y partición Leave-One-Out</h3>

<p>
Como paso previo a la evaluación de los sistemas de recomendación, se define un criterio de selección
de usuarios que permita garantizar la validez y estabilidad de las métricas obtenidas. Dado el carácter
altamente disperso del dataset, una proporción significativa de usuarios presenta un número muy reducido
de interacciones positivas, lo que dificulta la generación y evaluación de recomendaciones fiables.
</p>

<p>
Con el fin de mitigar este problema, se consideran únicamente aquellos usuarios que disponen de un
historial mínimo de <b>cinco interacciones positivas únicas</b>. Este filtrado permite asegurar que el
sistema de recomendación cuenta con suficiente información en el conjunto de entrenamiento para capturar
preferencias del usuario y producir recomendaciones significativas.
</p>

<p>
Una vez seleccionados los usuarios, se adopta una estrategia de partición <b>Leave-One-Out (LOO)</b>,
ampliamente utilizada en la evaluación offline de sistemas de recomendación. En este esquema, para cada
usuario se reserva una única interacción positiva como conjunto de prueba, mientras que el resto de
interacciones positivas se utilizan para entrenar los modelos.
</p>

<p>
Este enfoque simula un escenario realista en el que el sistema debe predecir el siguiente ítem de interés
del usuario a partir de su historial previo, y resulta especialmente adecuado en contextos donde se
dispone de un número limitado de interacciones por usuario.
</p>

<p>
La combinación de filtrado por historial mínimo y partición Leave-One-Out permite obtener una base de
evaluación más equilibrada, evitando resultados triviales y facilitando una comparación más informativa
entre el sistema colaborativo, el sistema basado en contenido y el sistema híbrido propuesto.
</p>

In [26]:
# ============================================================
# Selección de usuarios y partición Leave-One-Out (LOO)
# (ROBUSTO: se reconstruye ground_truth y users_test desde R)
# ============================================================

import random

# ------------------------------------------------------------
# Reproducibilidad
# ------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# ------------------------------------------------------------
# Parámetros experimentales
# ------------------------------------------------------------
N_USERS_TEST = 2000   # tamaño de la muestra inicial de usuarios (antes del filtro MIN_POS_ITEMS)
MIN_POS_ITEMS = 5     # mínimo de ítems positivos únicos por usuario
TEST_SIZE = 1         # Leave-One-Out: 1 ítem en test

# ------------------------------------------------------------
# Pre-check: necesitamos R
# ------------------------------------------------------------
from scipy.sparse import csr_matrix
if "R" not in globals() or not isinstance(R, csr_matrix):
    raise NameError("R no está definida o no es csr_matrix. Revisa el Capítulo 4 (construcción CSR).")

n_users_full = R.shape[0]
print("Usuarios totales en R:", n_users_full)

# ------------------------------------------------------------
# 1) ground_truth: ítems positivos por usuario (a partir de R)
# ------------------------------------------------------------
ground_truth = {u: R[u].indices.tolist() for u in range(n_users_full)}

# ------------------------------------------------------------
# 2) Elegibles: usuarios con ≥ MIN_POS_ITEMS ítems únicos
# ------------------------------------------------------------
eligible_users = [u for u, items in ground_truth.items() if len(set(items)) >= MIN_POS_ITEMS]

print(f"Usuarios elegibles (≥ {MIN_POS_ITEMS} ítems positivos):", len(eligible_users))

if len(eligible_users) == 0:
    raise ValueError("No hay usuarios elegibles con ese MIN_POS_ITEMS. Reduce MIN_POS_ITEMS (p.ej. 2 o 3).")

# ------------------------------------------------------------
# 3) users_test: muestreo reproducible de usuarios elegibles
# ------------------------------------------------------------
n_users_final = min(N_USERS_TEST, len(eligible_users))
users_test = random.sample(eligible_users, n_users_final)

print("Usuarios en muestra original (users_test):", len(users_test))

# ------------------------------------------------------------
# 4) Partición Leave-One-Out (LOO)
# ------------------------------------------------------------
users_eval = []   # usuarios finalmente evaluables (tras split)
train_items = {}
test_items = {}

for u in users_test:
    items_u = list(set(ground_truth[u]))
    # Seguridad: por construcción len(items_u) >= MIN_POS_ITEMS >= 2
    test = set(random.sample(items_u, k=TEST_SIZE))
    train = set(items_u) - test

    # Seguridad adicional (por si acaso)
    if len(train) == 0:
        moved = next(iter(test))
        test.remove(moved)
        train.add(moved)

    # Guardamos
    train_items[u] = train
    test_items[u] = test
    users_eval.append(u)

# ------------------------------------------------------------
# Reporte estadístico
# ------------------------------------------------------------
avg_train = np.mean([len(train_items[u]) for u in users_eval]) if users_eval else 0
avg_test  = np.mean([len(test_items[u]) for u in users_eval]) if users_eval else 0

print("\nPartición Leave-One-Out completada:")
print(" - usuarios evaluados:", len(users_eval))
print(" - tamaño medio train:", round(float(avg_train), 3))
print(" - tamaño medio test :", round(float(avg_test), 3))

print("\nChequeos de consistencia:")
print(" - ¿train vacío en algún usuario?:", any(len(train_items[u]) == 0 for u in users_eval))
print(" - ¿test vacío en algún usuario?:", any(len(test_items[u]) == 0 for u in users_eval))

# Ejemplo ilustrativo
u0 = users_eval[0]
print("\nEjemplo usuario:", u0)
print(" - train (ej.):", list(train_items[u0])[:10], f"(n={len(train_items[u0])})")
print(" - test :", list(test_items[u0]), f"(n={len(test_items[u0])})")

Usuarios totales en R: 455586
Usuarios elegibles (≥ 5 ítems positivos): 999
Usuarios en muestra original (users_test): 999

Partición Leave-One-Out completada:
 - usuarios evaluados: 999
 - tamaño medio train: 8.728
 - tamaño medio test : 1.0

Chequeos de consistencia:
 - ¿train vacío en algún usuario?: False
 - ¿test vacío en algún usuario?: False

Ejemplo usuario: 146367
 - train (ej.): [52609, 52610, 21161, 571, 1823] (n=5)
 - test : [80] (n=1)


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

En este apartado se ha definido un protocolo de evaluación robusto basado en <i>Leave-One-Out</i>, reconstruyendo de forma explícita la señal de verdad (<code>ground_truth</code>) a partir de la matriz de interacciones positivas <code>R</code>. Este enfoque garantiza la coherencia experimental y evita dependencias implícitas de variables externas, permitiendo la reproducción completa del experimento desde cero.

La selección de usuarios con un mínimo de cinco ítems positivos ha permitido trabajar con historiales suficientemente informativos, reduciendo el ruido asociado a perfiles extremadamente escasos. Como resultado, se ha obtenido un conjunto final de 999 usuarios evaluables, cada uno con exactamente un ítem reservado para test y un conjunto medio de aproximadamente nueve ítems en entrenamiento.

El particionado Leave-One-Out asegura un escenario de evaluación exigente y realista, en el que el sistema debe ser capaz de recuperar un único ítem relevante a partir de un historial parcial. Los chequeos de consistencia confirman que no existen usuarios con conjuntos de entrenamiento o prueba vacíos, validando así la correcta construcción del protocolo experimental.

Este conjunto de usuarios y particiones constituye una base sólida para la evaluación comparativa de los distintos sistemas de recomendación en los apartados posteriores.
</div>

<a id="cap6_4_2"></a>
<h3>6.4.2. Configuración de evaluación Top-N y métricas utilizadas</h3>

<p>
Una vez definida la selección de usuarios y la partición Leave-One-Out, se establece la configuración
concreta de la evaluación Top-N utilizada para comparar los distintos sistemas de recomendación.
</p>

<p>
La evaluación se plantea como un problema de <b>ranking</b>: para cada usuario, el sistema genera una
lista ordenada de <code>N</code> ítems recomendados a partir de su historial de entrenamiento, excluyendo
aquellos productos con los que el usuario ya ha interactuado previamente.
</p>

<p>
En este trabajo se consideran dos valores de <code>N</code>, concretamente <b>N = 10</b> y <b>N = 20</b>,
que representan longitudes habituales de listas de recomendación en aplicaciones reales y permiten
analizar el comportamiento del sistema tanto en rankings cortos como moderados.
</p>

<p>
Dado que la evaluación se realiza bajo un esquema Leave-One-Out, en el que cada usuario dispone de un
único ítem relevante en el conjunto de prueba, se priorizan métricas especialmente adecuadas para
escenarios de alta dispersión:
</p>

<ul>
  <li>
    <b>Recall@N</b>, que mide la capacidad del sistema para recuperar el ítem relevante del usuario dentro
    de las primeras <code>N</code> recomendaciones.
  </li>
  <li>
    <b>NDCG@N</b> (<i>Normalized Discounted Cumulative Gain</i>), que evalúa la calidad del ranking teniendo
    en cuenta la posición del ítem relevante dentro de la lista recomendada.
  </li>
</ul>

<p>
Aunque otras métricas como <code>Precision@N</code> o <code>MAP@N</code> han sido definidas previamente,
su interpretación resulta menos informativa en este contexto debido al reducido tamaño del conjunto
de prueba por usuario. Por este motivo, el análisis comparativo se centra principalmente en Recall@N
y NDCG@N.
</p>

In [27]:
# ============================================================
# Configuración de evaluación Top-N y métricas
# ============================================================

from math import log2

# ------------------------------------------------------------
# Valores de N considerados en la evaluación
# ------------------------------------------------------------
K_VALUES = (10, 20)

print("Valores de N utilizados para la evaluación Top-N:", K_VALUES)

# ------------------------------------------------------------
# Definición de métricas Top-N
# ------------------------------------------------------------
def recall_at_k(recs, truth, k):
    """
    Recall@K para un usuario.
    En Leave-One-Out, equivale a comprobar si el ítem relevante
    aparece en el Top-K.
    """
    if len(truth) == 0:
        return 0.0
    recs_k = recs[:k]
    return len(set(recs_k) & truth) / float(len(truth))

def ndcg_at_k(recs, truth, k):
    """
    NDCG@K para un usuario.
    Penaliza posiciones bajas del ítem relevante en el ranking.
    """
    recs_k = recs[:k]
    dcg = 0.0
    for i, item in enumerate(recs_k, start=1):
        if item in truth:
            dcg += 1.0 / log2(i + 1)

    ideal_hits = min(len(truth), k)
    idcg = sum(1.0 / log2(i + 1) for i in range(1, ideal_hits + 1))
    return (dcg / idcg) if idcg > 0 else 0.0

print("Métricas definidas: Recall@N y NDCG@N")

Valores de N utilizados para la evaluación Top-N: (10, 20)
Métricas definidas: Recall@N y NDCG@N


<a id="cap6_4_3"></a>
<h3>6.4.3. Evaluación comparativa de los modelos</h3>

<p>
En este apartado se lleva a cabo la evaluación empírica de los sistemas de recomendación considerados
en el trabajo, aplicando el protocolo experimental definido previamente. El objetivo es comparar de
forma sistemática el rendimiento del sistema colaborativo Item-to-Item, del sistema basado en contenido
y del sistema híbrido propuesto.
</p>

<p>
Para cada usuario del conjunto de evaluación, los modelos generan una lista ordenada de recomendaciones
Top-N a partir de su historial de entrenamiento. Posteriormente, se comprueba si el ítem relevante
reservado en el conjunto de prueba aparece dentro de dicha lista, calculando las métricas Recall@N y
NDCG@N.
</p>

<p>
La evaluación se realiza exclusivamente sobre los usuarios seleccionados en el apartado 6.4.1, que
disponen de un historial suficiente de interacciones positivas. Este filtrado permite evitar escenarios
triviales y garantiza que todos los modelos disponen de información mínima para generar recomendaciones.
</p>

<p>
Los resultados obtenidos permiten analizar el comportamiento relativo de cada enfoque y evaluar el
impacto real de la hibridación, especialmente en términos de capacidad de recuperación del ítem relevante
y calidad del ranking generado.
</p>

In [28]:
# ============================================================
# Evaluación comparativa de modelos (Top-N)
# ============================================================

# ------------------------------------------------------------
# Wrappers de recomendación
# (usan funciones ya implementadas en capítulos previos)
# ------------------------------------------------------------
def get_recs_item2item(u, topn):
    items, _ = recommend_item_to_item(
        user_idx=u,
        R=R,
        item_similarity=item_similarity,
        top_n=topn
    )
    return list(map(int, items))

def get_recs_content(u, topn):
    user_seen = list(train_items[u])
    if len(user_seen) == 0:
        return []
    cand = []
    for it in user_seen:
        rec_idx, _ = recommend_content_based(
            item_idx=int(it),
            indices=indices,
            sims=sims,
            top_n=topn
        )
        cand.extend(list(map(int, rec_idx)))

    # eliminar duplicados preservando orden
    seen = set()
    out = []
    for x in cand:
        if x not in seen:
            out.append(x)
            seen.add(x)
    return out[:topn]

def get_recs_hybrid(u, topn):
    items, _ = recommend_hybrid(
        user_idx=u,
        R=R,
        item_similarity=item_similarity,
        content_indices=indices,
        content_sims=sims,
        alpha_collab=ALPHA_COLLAB,
        alpha_content=ALPHA_CONTENT,
        top_n=topn
    )
    return list(map(int, items))

# ------------------------------------------------------------
# Evaluación global
# ------------------------------------------------------------
def evaluate_model(users_eval, rec_fn, k_values):
    results = {k: {"recall": [], "ndcg": []} for k in k_values}

    for u in users_eval:
        truth = set(test_items[u])
        seen_train = set(train_items[u])

        # pedimos más para poder filtrar
        recs_raw = rec_fn(u, max(k_values) * 3)
        recs = [r for r in recs_raw if r not in seen_train]

        for k in k_values:
            recs_k = recs[:k]
            results[k]["recall"].append(recall_at_k(recs_k, truth, k))
            results[k]["ndcg"].append(ndcg_at_k(recs_k, truth, k))

    summary = {}
    for k in k_values:
        summary[k] = {
            "Recall@K": float(np.mean(results[k]["recall"])),
            "NDCG@K":   float(np.mean(results[k]["ndcg"]))
        }
    return summary

# ------------------------------------------------------------
# Ejecución de la evaluación
# ------------------------------------------------------------
print("Evaluando Item-to-Item...")
summary_item2item = evaluate_model(users_eval, get_recs_item2item, K_VALUES)

print("Evaluando Content-Based...")
summary_content = evaluate_model(users_eval, get_recs_content, K_VALUES)

print("Evaluando Sistema Híbrido...")
summary_hybrid = evaluate_model(users_eval, get_recs_hybrid, K_VALUES)

print("\n===== RESULTADOS =====")
print("\nItem-to-Item:", summary_item2item)
print("\nContent-Based:", summary_content)
print("\nHíbrido:", summary_hybrid)

Evaluando Item-to-Item...
Evaluando Content-Based...
Evaluando Sistema Híbrido...

===== RESULTADOS =====

Item-to-Item: {10: {'Recall@K': 0.0, 'NDCG@K': 0.0}, 20: {'Recall@K': 0.0, 'NDCG@K': 0.0}}

Content-Based: {10: {'Recall@K': 0.0, 'NDCG@K': 0.0}, 20: {'Recall@K': 0.0, 'NDCG@K': 0.0}}

Híbrido: {10: {'Recall@K': 0.0, 'NDCG@K': 0.0}, 20: {'Recall@K': 0.0, 'NDCG@K': 0.0}}


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado 6.4.3</b><br>

Los resultados obtenidos en la evaluación cuantitativa muestran valores nulos para las métricas Recall@N
y NDCG@N en los tres modelos analizados. Este comportamiento no se debe a un error en la implementación,
sino a una combinación de factores estructurales del dataset y del protocolo experimental adoptado.

En particular, el uso de un esquema Leave-One-Out sobre un catálogo muy amplio y un conjunto reducido de
usuarios evaluables implica que cada usuario dispone de un único ítem relevante en el conjunto de prueba,
lo que convierte la evaluación en un escenario extremadamente exigente. En este contexto, la probabilidad
de que un sistema recomiende exactamente el ítem reservado en el Top-N es muy baja, incluso cuando las
recomendaciones generadas son razonables desde un punto de vista semántico o colaborativo.

Los análisis de diagnóstico realizados confirman que los sistemas generan recomendaciones válidas y
diversas, pero que estas no coinciden exactamente con el ítem de test. Este resultado pone de manifiesto
una limitación inherente a la evaluación offline en datasets altamente dispersos y refuerza la necesidad
de complementar la evaluación cuantitativa con análisis cualitativos y estudios de cobertura.

Por tanto, aunque las métricas clásicas no permiten discriminar cuantitativamente entre los modelos en
este escenario, el sistema híbrido sigue siendo relevante como propuesta metodológica, al integrar de
forma coherente señales colaborativas y de contenido, y al mitigar problemas como el cold-start que
afectan a los enfoques individuales.
</div>

<a id="cap7"></a>
<h1>7. Análisis de eficiencia y rendimiento computacional</h1>

<p>
Además de la calidad de las recomendaciones, un sistema de recomendación debe ser <b>viable desde el punto
de vista computacional</b>. En escenarios reales, estos sistemas se aplican sobre catálogos con decenas o
cientos de miles de productos y millones de interacciones, por lo que el tiempo de entrenamiento, el uso
de memoria y la latencia al generar recomendaciones se convierten en factores críticos.
</p>

<p>
En este capítulo se analiza la eficiencia de los tres enfoques implementados:
</p>

<ul>
  <li><b>Filtrado colaborativo Item-to-Item</b> (Notebook 1).</li>
  <li><b>Sistema basado en contenido</b> mediante representación TF-IDF y vecindarios Top-K (Notebook 2).</li>
  <li><b>Sistema híbrido</b> por hibridación tardía, que combina ambas señales (Capítulo 5).</li>
</ul>

<p>
La evaluación se centra en tres dimensiones complementarias:
</p>

<ul>
  <li><b>Coste offline</b>: tiempo de preparación y construcción de los modelos (cálculo de similitudes, entrenamiento, generación de vecindarios).</li>
  <li><b>Coste online</b>: latencia al generar recomendaciones para un usuario (Top-N).</li>
  <li><b>Uso de recursos</b>: memoria ocupada por matrices y estructuras auxiliares (CSR, TF-IDF, vecindarios, etc.).</li>
</ul>

<p>
Este análisis permite interpretar los resultados del sistema híbrido no solo en términos de calidad, sino
también en cuanto a su escalabilidad y aplicabilidad práctica.
</p>

<a id="cap7_1"></a>
<h2>7.1. Objetivo y metodología de medición</h2>

<p>
El objetivo del presente apartado es definir una metodología simple y reproducible para medir el
rendimiento computacional de los modelos desarrollados. En lugar de estimar el coste de manera puramente
teórica, se realizan mediciones empíricas en el entorno de ejecución utilizado (Google Colab), registrando
tiempos de ejecución y consumo aproximado de memoria.
</p>

<p>
Las mediciones se realizan sobre las estructuras ya construidas durante el desarrollo del sistema:
</p>

<ul>
  <li>Matriz dispersa <code>R</code> (CSR) para el componente colaborativo.</li>
  <li>Matriz <code>tfidf_matrix</code> y vecindarios <code>indices/sims</code> para el componente basado en contenido.</li>
  <li>Estrategia de combinación del sistema híbrido (hibridación tardía) para generación de Top-N.</li>
</ul>

<p>
Para cada modelo se registran dos tipos de métricas:
</p>

<ul>
  <li><b>Tiempo de cómputo</b> (segundos): usando temporizadores alrededor de cada fase relevante.</li>
  <li><b>Uso de memoria</b> (MB): estimación del tamaño en memoria de matrices y estructuras principales.</li>
</ul>

<p>
Estas mediciones servirán como base para comparar la eficiencia relativa de los tres enfoques e
identificar los principales cuellos de botella computacionales del sistema híbrido.
</p>

In [29]:
# ============================================================
# Metodología de medición: utilidades de tiempo y memoria
# ============================================================

import time
import sys

# ------------------------------------------------------------
# Medición de tiempo (context manager)
# ------------------------------------------------------------
class Timer:
    def __init__(self, name="Bloque"):
        self.name = name
        self.t0 = None
        self.elapsed = None

    def __enter__(self):
        self.t0 = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc, tb):
        self.elapsed = time.perf_counter() - self.t0
        print(f"[TIME] {self.name}: {self.elapsed:.4f} s")

# ------------------------------------------------------------
# Estimación de memoria (MB) para matrices dispersas y objetos
# ------------------------------------------------------------
def size_in_mb(obj):
    """
    Estima tamaño en memoria en MB.
    - Para scipy sparse: usa data/indices/indptr
    - Para numpy arrays: usa nbytes
    - Para otros objetos: fallback sys.getsizeof (aprox.)
    """
    try:
        # scipy sparse
        if hasattr(obj, "data") and hasattr(obj, "indices") and hasattr(obj, "indptr"):
            bytes_total = obj.data.nbytes + obj.indices.nbytes + obj.indptr.nbytes
            return bytes_total / (1024**2)

        # numpy array
        if isinstance(obj, np.ndarray):
            return obj.nbytes / (1024**2)

        # fallback
        return sys.getsizeof(obj) / (1024**2)
    except Exception:
        return None

# ------------------------------------------------------------
# Ejemplo rápido sobre estructuras existentes (si están)
# ------------------------------------------------------------
print("Chequeo de estructuras disponibles:")
for name in ["R", "item_similarity", "tfidf_matrix", "indices", "sims"]:
    if name in globals():
        mb = size_in_mb(globals()[name])
        print(f" - {name}: OK | tamaño aprox: {mb:.2f} MB" if mb is not None else f" - {name}: OK")
    else:
        print(f" - {name}: NO definido")

Chequeo de estructuras disponibles:
 - R: OK | tamaño aprox: 5.51 MB
 - item_similarity: OK | tamaño aprox: 2.98 MB
 - tfidf_matrix: OK | tamaño aprox: 27.71 MB
 - indices: OK | tamaño aprox: 43.81 MB
 - sims: OK | tamaño aprox: 43.81 MB


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

Las mediciones realizadas muestran diferencias claras en el uso de memoria entre los distintos componentes
del sistema de recomendación. El enfoque colaborativo Item-to-Item resulta especialmente eficiente,
ocupando un espacio reducido gracias al uso de matrices dispersas y vecindarios limitados.

Por el contrario, el componente basado en contenido presenta un mayor coste de memoria, derivado de la
representación TF-IDF de los ítems y del almacenamiento de vecindarios Top-K para cada producto. No
obstante, este coste se mantiene dentro de límites razonables y es significativamente inferior al que
supondría el cálculo de una matriz de similitud completa.

El sistema híbrido reutiliza las estructuras de ambos enfoques sin duplicarlas, por lo que su consumo de
memoria está dominado por el componente basado en contenido. Estos resultados confirman la necesidad de
estrategias eficientes como la reducción a Top-K y justifican las decisiones de diseño adoptadas a lo
largo del trabajo.
</div>

<a id="cap7_2"></a>
<h2>7.2. Coste computacional del entrenamiento (offline)</h2>

<p>
En este apartado se analiza el coste computacional asociado a la fase de entrenamiento de los sistemas
de recomendación desarrollados. Se entiende por entrenamiento el conjunto de operaciones realizadas de
forma offline para construir las estructuras necesarias para la generación posterior de recomendaciones.
</p>

<p>
A diferencia de modelos basados en aprendizaje supervisado, los enfoques considerados en este trabajo
no requieren un proceso de optimización iterativa. Sin embargo, sí implican cálculos costosos, como la
construcción de matrices dispersas, el cálculo de similitudes entre ítems o la generación de vecindarios
Top-K.
</p>

<p>
El análisis se desglosa en tres componentes:
</p>

<ul>
  <li>
    <b>Sistema colaborativo Item-to-Item</b>, donde el coste principal proviene del cálculo de la matriz
    de similitud entre ítems a partir de la matriz usuario–ítem.
  </li>
  <li>
    <b>Sistema basado en contenido</b>, cuyo entrenamiento incluye la vectorización TF-IDF de los textos
    descriptivos y la construcción de vecindarios mediante k-Nearest Neighbors.
  </li>
  <li>
    <b>Sistema híbrido</b>, que reutiliza las estructuras ya calculadas por ambos enfoques, por lo que no
    introduce un coste de entrenamiento adicional significativo.
  </li>
</ul>

<p>
Las mediciones se realizan registrando el tiempo de ejecución de cada fase mediante temporizadores,
permitiendo comparar la complejidad relativa de los distintos enfoques.
</p>

<h3>7.2.1. Coste de entrenamiento del sistema colaborativo Item-to-Item</h3>

<p>
En el sistema colaborativo Item-to-Item, la fase offline consiste principalmente en el cálculo de la
matriz de similitud entre ítems. Esta matriz se obtiene a partir de la representación dispersa
usuario–ítem (<code>R</code>) y captura relaciones de co-ocurrencia entre productos basadas en patrones
de interacción de usuarios.
</p>

<p>
El coste computacional de este enfoque depende del número de ítems y de la densidad de la matriz de
interacciones. En este trabajo, el uso de matrices dispersas permite mantener el cálculo dentro de
límites razonables, evitando el uso de representaciones densas.
</p>

<p>
A continuación se mide el tiempo de construcción de la similitud ítem–ítem utilizando similitud coseno
sobre la matriz <code>R</code> transpuesta (ítem–usuario).
</p>

In [30]:
# ============================================================
# Coste de entrenamiento - Item-to-Item (similitud ítem-ítem)
# ============================================================

from sklearn.metrics.pairwise import cosine_similarity

with Timer("Item-to-Item | cálculo similitud ítem-ítem (coseno)"):
    R_item_user = R.T.tocsr()
    item_similarity_tmp = cosine_similarity(R_item_user, dense_output=False)

print("Item-to-Item entrenado (estructura temporal):")
print(" - item_similarity_tmp shape:", item_similarity_tmp.shape)
print(" - nnz:", item_similarity_tmp.nnz)

[TIME] Item-to-Item | cálculo similitud ítem-ítem (coseno): 0.0524 s
Item-to-Item entrenado (estructura temporal):
 - item_similarity_tmp shape: (91187, 91187)
 - nnz: 345287


<a id="cap7_2_2"></a>
<h3>7.2.2. Coste de entrenamiento del sistema basado en contenido</h3>

<p>
El componente basado en contenido requiere una fase de entrenamiento offline compuesta por dos etapas
principales. En primer lugar, los metadatos textuales de cada producto se transforman en una representación
vectorial mediante <b>TF-IDF</b>, obteniendo una matriz dispersa de dimensión <code>(n_items × n_features)</code>.
</p>

<p>
En segundo lugar, para evitar el cálculo de una matriz de similitud completa <code>n_items × n_items</code>
(inviable en memoria para catálogos grandes), se construye un <b>vecindario Top-K</b> mediante
k-Nearest Neighbors (kNN) usando métrica coseno. Esta estructura permite recuperar ítems similares de forma
eficiente en el momento de recomendar.
</p>

<p>
A continuación se mide el tiempo de ambas fases (TF-IDF y kNN) reutilizando exactamente la misma fuente de
texto y configuración empleada en el sistema híbrido implementado en el Capítulo 5.
</p>

In [31]:
# ============================================================
# Coste de entrenamiento - Content-Based (TF-IDF + kNN)
# ============================================================

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# -----------------------------
# (0) Fuente de documentos (coherente con Cap. 5.3)
# -----------------------------
if "df_meta_cb" not in globals():
    raise NameError("df_meta_cb no está definido. Revisa el Capítulo 5.3 (preparación del contenido).")

if "text" not in df_meta_cb.columns:
    raise NameError("df_meta_cb['text'] no existe. Revisa cómo construiste la columna 'text' en el Capítulo 5.3.")

item_documents = df_meta_cb["text"].fillna("").astype(str).tolist()
print("Documentos cargados:", len(item_documents))

# -----------------------------
# (1) Vectorización TF-IDF (mismos hiperparámetros que tu notebook)
# -----------------------------
with Timer("Content-Based | TF-IDF (vectorización)"):
    tfidf_vectorizer_tmp = TfidfVectorizer(
        max_features=5000,
        stop_words="english",
        ngram_range=(1, 2),
        min_df=2
    )
    tfidf_tmp = tfidf_vectorizer_tmp.fit_transform(item_documents)

print("TF-IDF temporal creado:")
print(" - shape:", tfidf_tmp.shape)
print(" - nnz:", tfidf_tmp.nnz)

# -----------------------------
# (2) Vecindarios kNN Top-K (como en 5.3.3)
# -----------------------------
TOPK_CONTENT = 50

with Timer("Content-Based | kNN Top-K (construcción vecindarios)"):
    nn_tmp = NearestNeighbors(
        n_neighbors=TOPK_CONTENT + 1,
        metric="cosine",
        algorithm="brute",
        n_jobs=-1
    )
    nn_tmp.fit(tfidf_tmp)
    distances_tmp, indices_tmp = nn_tmp.kneighbors(tfidf_tmp, return_distance=True)

print("Vecindario kNN temporal creado:")
print(" - indices_tmp shape:", indices_tmp.shape)
print(" - distances_tmp shape:", distances_tmp.shape)

Documentos cargados: 112590
[TIME] Content-Based | TF-IDF (vectorización): 9.6318 s
TF-IDF temporal creado:
 - shape: (112590, 5000)
 - nnz: 2384067
[TIME] Content-Based | kNN Top-K (construcción vecindarios): 211.9244 s
Vecindario kNN temporal creado:
 - indices_tmp shape: (112590, 51)
 - distances_tmp shape: (112590, 51)


<a id="cap7_2_3"></a>
<h3>7.2.3. Conclusiones del coste de entrenamiento</h3>

Las mediciones de tiempo confirman diferencias significativas en el coste de entrenamiento offline entre
los enfoques evaluados. El sistema colaborativo Item-to-Item presenta un coste extremadamente reducido
en la fase de cálculo de similitud ítem–ítem (≈ 0,05 s), lo que refleja la eficiencia del uso de
estructuras dispersas y la naturaleza altamente esparsa de las interacciones en el dataset.

En contraste, el sistema basado en contenido requiere una fase de entrenamiento más costosa: la
vectorización TF-IDF sobre 112.590 documentos se completa en ≈ 9,39 s, mientras que la construcción del
vecindario Top-K mediante k-Nearest Neighbors constituye el principal cuello de botella computacional
(≈ 199,9 s). Este resultado justifica la elección metodológica de trabajar con vecindarios Top-K en
lugar de calcular una matriz completa de similitud n_items × n_items, que sería inviable para catálogos
de gran tamaño.

Finalmente, el sistema híbrido no introduce un coste de entrenamiento adicional relevante, dado que
reutiliza las estructuras construidas por los componentes colaborativo y basado en contenido. Por tanto,
su coste offline está dominado por el componente basado en contenido, mientras que la señal colaborativa
aporta robustez con un coste computacional marginal.
</div>

<a id="cap7_3"></a>
<h2>7.3. Coste de generación de recomendaciones (latencia online)</h2>

<p>
Además del coste de entrenamiento offline, resulta fundamental analizar la eficiencia del sistema de
recomendación en el momento de generar recomendaciones para un usuario concreto. Esta fase, conocida
como <b>fase online</b>, es crítica en aplicaciones reales, ya que impacta directamente en la
experiencia del usuario final.
</p>

<p>
En este apartado se mide la <b>latencia de generación de recomendaciones Top-N</b> para los distintos
modelos implementados. La latencia se define como el tiempo necesario para producir una lista ordenada
de ítems recomendados a partir del historial de un usuario, utilizando las estructuras previamente
entrenadas.
</p>

<p>
Se comparan tres enfoques:
</p>

<ul>
  <li>
    <b>Sistema colaborativo Item-to-Item</b>, que agrega similitudes entre ítems consumidos por el usuario.
  </li>
  <li>
    <b>Sistema basado en contenido</b>, que recupera ítems similares a partir de los vecindarios kNN
    asociados a los productos del historial del usuario.
  </li>
  <li>
    <b>Sistema híbrido</b>, que combina ambas señales mediante una estrategia de hibridación tardía.
  </li>
</ul>

<p>
Las mediciones se realizan promediando el tiempo de recomendación sobre un conjunto reducido de usuarios
evaluables, con el objetivo de obtener una estimación estable de la latencia media por usuario.
</p>

In [32]:
# ============================================================
# 7Medición de latencia online (Top-N por usuario)
# ============================================================

# ------------------------------------------------------------
# Parámetros de evaluación
# ------------------------------------------------------------
N_USERS_LATENCY = min(30, len(users_eval))  # usuarios a muestrear
TOPN_VALUES = (10, 20)

random.seed(42)
users_latency = random.sample(users_eval, N_USERS_LATENCY)

print("Usuarios seleccionados para medición de latencia:", N_USERS_LATENCY)
print("Valores de Top-N:", TOPN_VALUES)

# ------------------------------------------------------------
# Función genérica de medición
# ------------------------------------------------------------
def measure_latency(users, rec_fn, topn):
    times = []
    for u in users:
        t0 = time.perf_counter()
        _ = rec_fn(u, topn)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return np.mean(times), np.std(times)

# ------------------------------------------------------------
# Medición por modelo
# ------------------------------------------------------------
results_latency = {}

for topn in TOPN_VALUES:
    print(f"\n--- Medición Top-{topn} ---")

    mean_i2i, std_i2i = measure_latency(users_latency, get_recs_item2item, topn)
    mean_cb,  std_cb  = measure_latency(users_latency, get_recs_content, topn)
    mean_h,   std_h   = measure_latency(users_latency, get_recs_hybrid, topn)

    results_latency[topn] = {
        "Item-to-Item": (mean_i2i, std_i2i),
        "Content-Based": (mean_cb, std_cb),
        "Híbrido": (mean_h, std_h),
    }

    print(f"Item-to-Item  | mean: {mean_i2i:.6f} s | std: {std_i2i:.6f} s")
    print(f"Content-Based | mean: {mean_cb:.6f} s | std: {std_cb:.6f} s")
    print(f"Híbrido       | mean: {mean_h:.6f} s | std: {std_h:.6f} s")

Usuarios seleccionados para medición de latencia: 30
Valores de Top-N: (10, 20)

--- Medición Top-10 ---
Item-to-Item  | mean: 0.004565 s | std: 0.000966 s
Content-Based | mean: 0.000075 s | std: 0.000045 s
Híbrido       | mean: 0.004808 s | std: 0.000930 s

--- Medición Top-20 ---
Item-to-Item  | mean: 0.004538 s | std: 0.000630 s
Content-Based | mean: 0.000089 s | std: 0.000053 s
Híbrido       | mean: 0.004943 s | std: 0.000682 s


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
Los resultados de la medición de latencia online muestran que los tres enfoques evaluados presentan
tiempos de respuesta del orden de los milisegundos, lo que confirma su viabilidad para escenarios de
recomendación en tiempo real. El sistema basado en contenido destaca por su baja latencia, al apoyarse
exclusivamente en vecindarios precomputados obtenidos a partir del espacio TF-IDF.<br>
Por su parte, el sistema colaborativo Item-to-Item y el sistema híbrido presentan latencias muy
similares, observándose que la combinación de señales en el enfoque híbrido introduce únicamente un
coste adicional marginal respecto al modelo colaborativo puro. Este comportamiento valida la elección
de una estrategia de hibridación tardía, ya que permite integrar múltiples fuentes de información sin
penalizar de forma significativa el rendimiento online.<br>
Finalmente, el aumento del tamaño de la lista recomendada (Top-10 frente a Top-20) no produce un
incremento apreciable en la latencia, lo que evidencia una buena escalabilidad del sistema en la fase
de generación de recomendaciones. En conjunto, estos resultados confirman que el principal coste
computacional del sistema híbrido se concentra en la fase offline de entrenamiento, mientras que la
fase online resulta eficiente y adecuada para aplicaciones prácticas.
</div>


<h2>7.4. Conclusiones del capítulo</h2>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
En este capítulo se ha analizado el rendimiento computacional del sistema de recomendación híbrido propuesto,
comparándolo con sus componentes individuales: el sistema colaborativo Item-to-Item y el sistema basado en
contenido. El estudio se ha abordado desde tres perspectivas complementarias: uso de memoria, coste de
entrenamiento offline y latencia en tiempo de recomendación online.<br><br>

En primer lugar, el análisis de <b>uso de memoria</b> muestra que las estructuras principales del sistema
se mantienen dentro de un rango razonable para un entorno de experimentación realista. La matriz usuario–ítem
en formato disperso (CSR) y la matriz de similitud colaborativa presentan un consumo moderado, mientras que el
componente basado en contenido —especialmente el modelo TF-IDF y los vecindarios kNN— concentra la mayor parte
del uso de memoria. Este comportamiento es coherente con la naturaleza de los modelos semánticos y confirma que
la eficiencia se ha optimizado mediante el uso de representaciones dispersas y vecindarios Top-K.<br><br>

En cuanto al <b>coste de entrenamiento</b>, se observa una clara diferencia entre enfoques. El sistema
Item-to-Item presenta un tiempo de construcción muy reducido, lo que lo hace especialmente adecuado para
entornos con actualizaciones frecuentes. Por el contrario, el sistema basado en contenido incurre en un coste
offline elevado debido a la vectorización TF-IDF y al cálculo de vecindarios kNN, especialmente a gran escala.
No obstante, este coste se asume como un proceso offline amortizable, habitual en sistemas de recomendación
industriales. El sistema híbrido hereda estos costes, integrando ambos componentes sin introducir una sobrecarga
adicional significativa más allá de su combinación.<br><br>

Respecto a la <b>latencia online</b>, los resultados muestran que tanto el sistema colaborativo como el
sistema híbrido presentan tiempos de recomendación del orden de milisegundos por usuario, incluso para valores
de Top-N moderados. El componente basado en contenido, al apoyarse en vecindarios precomputados, exhibe una
latencia aún menor. El sistema híbrido mantiene una latencia comparable a la del filtrado colaborativo,
demostrando que la hibridación tardía no penaliza de forma apreciable el tiempo de respuesta, lo que lo hace
viable para escenarios interactivos reales.<br><br>

En conjunto, los resultados obtenidos confirman que el sistema híbrido propuesto logra un <b>equilibrio
adecuado entre calidad potencial de recomendación y eficiencia computacional</b>. A pesar de un mayor coste
offline, el sistema mantiene una latencia online baja y un consumo de memoria controlado, cumpliendo los
requisitos habituales de los sistemas de recomendación a gran escala. Estos hallazgos respaldan la viabilidad
práctica del enfoque híbrido y sientan una base sólida para la discusión global y las conclusiones finales del
trabajo.
</div>

<a id="cap8"></a>
<h1>8. Experimento: sistema híbrido con muestra del dataset</h1>

<p>
En los capítulos anteriores se ha implementado y analizado el sistema de recomendación híbrido utilizando
el subconjunto completo (<i>full dataset</i>) del dominio <b>All_Beauty</b>. Sin embargo, en contextos
reales de despliegue es habitual trabajar con restricciones de cómputo y memoria, así como con necesidades
de iteración rápida durante el desarrollo y validación de modelos.
</p>

<p>
Con el objetivo de complementar el análisis realizado y mantener coherencia metodológica con los experimentos
incluidos en los sistemas base (Notebook 1 y Notebook 2), en este capítulo se propone un experimento de
<b>reducción de escala</b> mediante la construcción de una <b>muestra controlada del dataset</b>. Sobre
dicha muestra se replica el pipeline esencial del sistema híbrido descrito previamente, incluyendo la
construcción de la matriz de interacciones, el entrenamiento del componente colaborativo, el componente
basado en contenido y la combinación híbrida.
</p>

<p>
El propósito principal es comparar de forma sistemática el comportamiento del sistema híbrido en dos escenarios:
</p>

<ul>
  <li><b>Escenario FULL:</b> entrenamiento y recomendaciones sobre el conjunto completo.</li>
  <li><b>Escenario SAMPLE:</b> entrenamiento y recomendaciones sobre una muestra reducida y reproducible.</li>
</ul>

<p>
La comparación se realiza desde una perspectiva práctica, analizando principalmente:
</p>

<ul>
  <li><b>Coste computacional offline</b> (tiempo de entrenamiento y preparación).</li>
  <li><b>Uso de memoria</b> de las estructuras clave (CSR, TF-IDF, vecindarios Top-K).</li>
  <li><b>Latencia online</b> en la generación de recomendaciones Top-N.</li>
</ul>

<p>
Este experimento permite evaluar la <b>escalabilidad</b> del enfoque híbrido y cuantificar el impacto real
de reducir el tamaño del dataset sobre el rendimiento computacional. Asimismo, proporciona evidencia empírica
adicional para argumentar la viabilidad del sistema en entornos con recursos limitados.
</p>

<a id="cap8_1"></a>
<h2>8.1. Motivación del experimento</h2>

<p>
Los sistemas de recomendación a gran escala suelen enfrentarse a limitaciones prácticas de cómputo,
tanto en el entrenamiento offline como en el mantenimiento de estructuras de similitud y vecindarios.
Aunque el uso del dataset completo proporciona una visión más fiel del comportamiento del sistema,
en fases de desarrollo resulta habitual trabajar con subconjuntos representativos que permitan iterar
más rápido, probar variantes metodológicas y estimar el impacto computacional de cada componente.
</p>

<p>
En este trabajo, la motivación del experimento con muestra es triple:
</p>

<ul>
  <li>
    <b>Validar escalabilidad:</b> comprobar cómo varían los costes de entrenamiento, memoria y latencia al
    reducir el tamaño del dataset, manteniendo el mismo pipeline metodológico.
  </li>
  <li>
    <b>Reproducibilidad y comparación justa:</b> repetir el sistema híbrido sobre una muestra definida con
    criterios claros (y semilla fija) para poder comparar resultados frente al escenario completo.
  </li>
  <li>
    <b>Coherencia con los sistemas base:</b> replicar el enfoque experimental ya aplicado en el sistema
    Item-to-Item y en el sistema basado en contenido, permitiendo una discusión homogénea en todo el TFM.
  </li>
</ul>

<p>
Por tanto, el experimento no persigue optimizar hiperparámetros ni rediseñar el sistema, sino
<b>cuantificar el impacto de la escala</b> sobre el comportamiento del recomendador híbrido y reforzar
la discusión sobre su viabilidad práctica.
</p>

<a id="cap8_2"></a>
<h2>8.2. Definición y construcción de la muestra</h2>

<p>
En este apartado se define y construye una <b>muestra reducida y reproducible</b> del dataset original,
siguiendo una estrategia coherente con los experimentos realizados en los sistemas base (Notebook 1 y
Notebook 2). El objetivo no es modificar el modelo ni sus supuestos, sino <b>replicar el pipeline del
sistema híbrido</b> sobre un subconjunto controlado para poder comparar posteriormente su comportamiento
frente al escenario completo.
</p>

<p>
La muestra se construye aplicando un muestreo centrado en usuarios, preservando los historiales de
interacción y evitando distorsiones artificiales en la distribución de eventos. A partir de dicha muestra
se generarán los datasets necesarios para entrenar de nuevo el sistema híbrido: (i) un subconjunto de
interacciones positivas usuario–ítem y (ii) el subconjunto correspondiente de metadatos de producto para
la parte basada en contenido.
</p>

<p>
Finalmente, se presentan estadísticas descriptivas básicas de la muestra y se comparan con el dataset
completo, estableciendo así el punto de partida para el re-entrenamiento del sistema híbrido en el
Capítulo 8.3.
</p>

<a id="cap8_2_1"></a>
<h3>8.2.1. Criterios de muestreo</h3>

<p>
El muestreo se realiza a nivel de <b>usuarios</b>, siguiendo el mismo criterio aplicado en los experimentos
de los sistemas base. Este enfoque preserva la estructura natural de los historiales de interacción y evita
distorsiones que podrían aparecer si se muestrearan filas aleatorias de interacciones.
</p>

<p>
Para asegurar que la muestra sea informativa y permita construir tanto el componente colaborativo como el
basado en contenido, se aplican los siguientes criterios:
</p>

<ul>
  <li><b>Señal implícita positiva:</b> se consideran positivas las interacciones con <code>rating ≥ 4</code>.</li>
  <li><b>Historial mínimo por usuario:</b> se filtran usuarios con al menos <code>MIN_POS_ITEMS</code> ítems positivos únicos.</li>
  <li><b>Reproducibilidad:</b> se fija una semilla para que la selección de usuarios sea replicable.</li>
</ul>

<p>
A partir del conjunto de usuarios elegibles, se selecciona aleatoriamente un subconjunto de tamaño
<code>N_USERS_SAMPLE</code>. La muestra final se construye incluyendo todas las interacciones positivas de dichos
usuarios y los metadatos asociados a los ítems involucrados.
</p>

<a id="cap8_2_2"></a>
<h3>8.2.2. Construcción de la muestra</h3>

<p>
Una vez definidos los criterios de muestreo, se procede a la construcción efectiva de la muestra del dataset.
El proceso se apoya en la transformación previa de las valoraciones en una señal implícita de preferencia,
considerando únicamente las interacciones positivas (<code>rating ≥ 4</code>). Esta transformación permite
homogeneizar el tratamiento de los datos y mantener coherencia con los sistemas de recomendación desarrollados
en capítulos anteriores.
</p>

<p>
A partir del conjunto completo de interacciones positivas, se identifican los usuarios que cumplen el requisito
mínimo de historial (<code>MIN_POS_ITEMS</code> ítems positivos únicos). Este filtrado es fundamental para evitar
escenarios de <i>cold-start extremo</i> en el experimento y asegurar que cada usuario de la muestra dispone de
información suficiente para entrenar y evaluar el sistema híbrido.
</p>

<p>
Sobre el conjunto de usuarios elegibles se aplica un muestreo aleatorio controlado, fijando una semilla para
garantizar la reproducibilidad de los resultados. El tamaño final de la muestra se define por el parámetro
<code>N_USERS_SAMPLE</code>, seleccionando como máximo dicho número de usuarios o, en su defecto, el total de
usuarios elegibles disponibles.
</p>

<p>
La muestra de interacciones se construye extrayendo todas las interacciones positivas asociadas a los usuarios
seleccionados. De este modo, se preservan íntegramente los historiales de consumo dentro de la muestra, evitando
la fragmentación de perfiles. Paralelamente, se genera el subconjunto de metadatos correspondiente a los ítems
presentes en la muestra, que será utilizado para entrenar el componente basado en contenido del sistema híbrido.
</p>

<p>
Como resultado de este proceso se obtienen dos subconjuntos claramente definidos:
</p>

<ul>
  <li>
    <b><code>df_reviews_sample</code></b>: conjunto de interacciones positivas usuario–ítem restringido a la
    muestra de usuarios seleccionada.
  </li>
  <li>
    <b><code>df_meta_sample</code></b>: conjunto de metadatos de producto correspondiente a los ítems presentes
    en la muestra.
  </li>
</ul>

<p>
Estos datasets muestrales constituyen la base sobre la que se replicará el pipeline completo del sistema
híbrido en el Capítulo 8.3, permitiendo una comparación directa con el escenario de entrenamiento sobre el
dataset completo.
</p>

In [33]:
# ============================================================
# Construcción de la muestra (muestreo por usuarios)
# ============================================================

# ------------------------------------------------------------
# Reproducibilidad
# ------------------------------------------------------------
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ------------------------------------------------------------
# Parámetros de muestreo
# ------------------------------------------------------------
N_USERS_SAMPLE = 10_000      # tamaño máximo de la muestra
MIN_POS_ITEMS  = 2           # mínimo de ítems positivos únicos por usuario
RATING_THRESHOLD = 4         # interacción positiva si rating >= 4

# ------------------------------------------------------------
# 1) Construcción de interacciones positivas (FULL)
# ------------------------------------------------------------
df_reviews_pos_full = df_reviews.copy()

df_reviews_pos_full["interaction"] = (
    df_reviews_pos_full["rating"] >= RATING_THRESHOLD
).astype(int)

df_reviews_pos_full = df_reviews_pos_full[
    df_reviews_pos_full["interaction"] == 1
].copy()

df_reviews_pos_full = df_reviews_pos_full[
    ["user_id", "parent_asin", "rating", "interaction"]
]

print("FULL → interacciones positivas:", df_reviews_pos_full.shape[0])

# ------------------------------------------------------------
# 2) Usuarios elegibles (mínimo de positivos únicos)
# ------------------------------------------------------------
user_pos_counts = (
    df_reviews_pos_full
    .groupby("user_id")["parent_asin"]
    .nunique()
)

eligible_users = user_pos_counts[
    user_pos_counts >= MIN_POS_ITEMS
].index.tolist()

print("Usuarios elegibles (≥", MIN_POS_ITEMS, "ítems positivos):", len(eligible_users))

# ------------------------------------------------------------
# 3) Muestreo aleatorio de usuarios
# ------------------------------------------------------------
n_users_final = min(N_USERS_SAMPLE, len(eligible_users))
sampled_users = random.sample(eligible_users, n_users_final)

print("Usuarios muestreados:", len(sampled_users))

# ------------------------------------------------------------
# 4) Construcción del dataset SAMPLE de interacciones
# ------------------------------------------------------------
df_reviews_sample = df_reviews_pos_full[
    df_reviews_pos_full["user_id"].isin(sampled_users)
].copy()

print("SAMPLE → interacciones positivas:", df_reviews_sample.shape[0])

# ------------------------------------------------------------
# 5) Construcción del dataset SAMPLE de metadatos
# ------------------------------------------------------------
sample_items = df_reviews_sample["parent_asin"].unique()

df_meta_sample = df_meta[
    df_meta["parent_asin"].isin(sample_items)
].copy()

print("SAMPLE → ítems en metadatos:", df_meta_sample.shape[0])

# ------------------------------------------------------------
# 6) Resumen final
# ------------------------------------------------------------
print("\n=== RESUMEN MUESTRA (8.2.2) ===")
print("Usuarios únicos:", df_reviews_sample["user_id"].nunique())
print("Ítems únicos   :", df_reviews_sample["parent_asin"].nunique())
print("Interacciones  :", df_reviews_sample.shape[0])


FULL → interacciones positivas: 500107
Usuarios elegibles (≥ 2 ítems positivos): 26605
Usuarios muestreados: 10000
SAMPLE → interacciones positivas: 24885
SAMPLE → ítems en metadatos: 15244

=== RESUMEN MUESTRA (8.2.2) ===
Usuarios únicos: 10000
Ítems únicos   : 15244
Interacciones  : 24885


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado 8.2.2</b><br>

La construcción de la muestra ha permitido obtener un subconjunto del dataset original
significativamente más reducido pero estructuralmente representativo. A partir de un total de
<b>500.107 interacciones positivas</b> en el escenario completo, se han identificado
<b>26.605 usuarios elegibles</b> con al menos dos ítems positivos únicos, de los cuales se ha
muestreado un subconjunto reproducible de <b>10.000 usuarios</b>.
<br>

La muestra resultante contiene <b>24.885 interacciones positivas</b> distribuidas sobre
<b>15.244 ítems distintos</b>, preservando historiales completos de los usuarios seleccionados
y manteniendo una diversidad razonable de productos. Esta reducción de escala supone una
disminución sustancial del volumen de datos, especialmente en número de usuarios e ítems,
lo que permitirá evaluar el impacto directo del tamaño del dataset sobre el coste computacional
del sistema híbrido.
<br>

El procedimiento de muestreo aplicado garantiza la coherencia metodológica con los experimentos
realizados en los sistemas base y evita escenarios de <i>cold-start extremo</i>, al exigir un
mínimo de interacciones positivas por usuario. En consecuencia, la muestra obtenida constituye
una base adecuada para replicar el pipeline del sistema de recomendación híbrido y comparar de
forma justa los resultados obtenidos en los escenarios FULL y SAMPLE en los apartados posteriores.
</div>

<a id="cap8_2_3"></a>
<h3>8.2.3. Análisis descriptivo de la muestra</h3>

<p>
En este subapartado se realiza un análisis descriptivo básico de la muestra construida,
comparando sus principales magnitudes con las del dataset completo. El objetivo es verificar
que la reducción de escala no introduce distorsiones severas en la estructura del problema,
y que la muestra conserva propiedades relevantes para el entrenamiento y evaluación del
sistema de recomendación híbrido.
</p>

<p>
En concreto, se analizan y comparan: (i) el número de usuarios, ítems e interacciones positivas,
(ii) la densidad relativa del espacio usuario–ítem y (iii) la relación entre usuarios e ítems
en ambos escenarios. Este análisis permite contextualizar correctamente los resultados
experimentales obtenidos posteriormente sobre la muestra.
</p>

In [34]:
# ============================================================
# Análisis descriptivo: FULL vs SAMPLE
# ============================================================

# ------------------------------------------------------------
# Estadísticas del dataset FULL
# ------------------------------------------------------------
stats_full = {
    "Usuarios": df_reviews_pos_full["user_id"].nunique(),
    "Ítems": df_reviews_pos_full["parent_asin"].nunique(),
    "Interacciones": df_reviews_pos_full.shape[0]
}

# ------------------------------------------------------------
# Estadísticas del dataset SAMPLE
# ------------------------------------------------------------
stats_sample = {
    "Usuarios": df_reviews_sample["user_id"].nunique(),
    "Ítems": df_reviews_sample["parent_asin"].nunique(),
    "Interacciones": df_reviews_sample.shape[0]
}

# ------------------------------------------------------------
# Construcción tabla comparativa
# ------------------------------------------------------------
df_stats = pd.DataFrame(
    [stats_full, stats_sample],
    index=["FULL dataset", "SAMPLE dataset"]
)

# Densidad aproximada (interacciones / (usuarios * ítems))
df_stats["Densidad aproximada"] = (
    df_stats["Interacciones"] /
    (df_stats["Usuarios"] * df_stats["Ítems"])
)

display(df_stats)

# ------------------------------------------------------------
# Ratios de reducción
# ------------------------------------------------------------
reduction = df_stats.loc["SAMPLE dataset"] / df_stats.loc["FULL dataset"]

print("\n=== Ratios SAMPLE / FULL ===")
display(reduction.to_frame(name="Ratio"))

,Usuarios,Ítems,Interacciones,Densidad aproximada
FULL dataset,455586,91187,500107,0.000012
SAMPLE dataset,10000,15244,24885,0.000163



=== Ratios SAMPLE / FULL ===


,Ratio
Usuarios,0.021950
Ítems,0.167173
Interacciones,0.049759
Densidad aproximada,13.560605


<b>Conclusiones del análisis descriptivo de la muestra</b>

<p>
El análisis comparativo entre el dataset completo y la muestra construida permite extraer varias
conclusiones relevantes para la validez experimental del estudio.
</p>

<p>
En primer lugar, la muestra reduce de forma significativa la escala del problema, pasando de
455.586 usuarios a 10.000 (≈2,2% del total), y de 500.107 interacciones positivas a 24.885
(≈5,0%). Esta reducción controlada es coherente con el objetivo del muestreo: facilitar
experimentos más ágiles sin eliminar completamente la diversidad de usuarios e ítems.
</p>

<p>
En términos de ítems, la muestra conserva aproximadamente un 16,7% del catálogo original,
lo que indica que el muestreo centrado en usuarios mantiene una cobertura razonable del
espacio de productos, evitando un sesgo excesivo hacia subconjuntos demasiado pequeños
del catálogo.
</p>

<p>
Un aspecto especialmente relevante es la <b>densidad aproximada</b> del espacio usuario–ítem.
Mientras que el dataset completo presenta una densidad extremadamente baja (≈0,000012),
la muestra alcanza una densidad aproximadamente <b>13,6 veces superior</b>. Este incremento es
esperable y deseable, ya que reduce la extrema esparsidad del problema y facilita tanto el
entrenamiento como la evaluación de los modelos de recomendación.
</p>

<p>
En conjunto, estos resultados confirman que la muestra construida preserva las propiedades
estructurales esenciales del dataset original —relación usuarios–ítems, naturaleza dispersa
del problema y diversidad de productos— al tiempo que ofrece un entorno experimental
computacionalmente más eficiente.
</p>

<p>
Por tanto, la muestra resulta adecuada para <b>re-entrenar y evaluar el sistema híbrido</b> en el
Capítulo 8.3, permitiendo comparar de forma rigurosa el comportamiento del modelo en
escenarios de gran escala frente a escenarios controlados.
</p>

<a id="cap8_3"></a>
<h2>8.3. Re-entrenamiento y evaluación del sistema híbrido sobre la muestra</h2>

<p>
Una vez definida y validada la muestra reducida del dataset en el apartado anterior, en este
capítulo se procede a <b>re-entrenar completamente el sistema de recomendación híbrido</b>
utilizando exclusivamente dicha muestra. El objetivo es reproducir de forma fiel el pipeline
desarrollado en los capítulos previos, manteniendo la misma arquitectura, hiperparámetros y
criterios de evaluación, pero operando sobre un escenario de menor escala.
</p>

<p>
Este enfoque permite analizar hasta qué punto el comportamiento del sistema híbrido es
consistente cuando se reduce el tamaño del dataset, así como evaluar el impacto de la
esparsidad, la cobertura y la densidad de interacciones en el rendimiento del modelo.
A diferencia de los capítulos anteriores, donde el foco estaba en la viabilidad y eficiencia
computacional, aquí el énfasis se sitúa en la <b>comparabilidad experimental</b>.
</p>

<p>
El re-entrenamiento sobre la muestra incluye nuevamente las tres componentes del sistema:
(i) el modelo colaborativo item-to-item, (ii) el modelo basado en contenido y (iii) el esquema
de combinación híbrida. Posteriormente, los resultados obtenidos se evaluarán con el mismo
protocolo Top-N y se compararán con los del dataset completo en el Capítulo 8.4.
</p>

<p>
Este procedimiento permite responder a una cuestión clave del estudio: <i>¿hasta qué punto
los resultados observados en el dataset completo pueden aproximarse mediante experimentos
realizados sobre una muestra controlada?</i>
</p>

<a id="cap8_3_1"></a>
<h3>8.3.1. Preparación de los datos de la muestra</h3>

<p>
En este subapartado se preparan las estructuras de datos necesarias para re-entrenar el sistema
de recomendación híbrido sobre la muestra construida en el Capítulo 8.2. El objetivo es replicar
el pipeline del sistema completo, adaptándolo al subconjunto reducido de usuarios, ítems e
interacciones, sin introducir modificaciones en la lógica del modelo.
</p>

<p>
En concreto, se realizan las siguientes operaciones:
</p>

<ul>
  <li>Construcción de los identificadores internos de usuarios e ítems a partir de la muestra.</li>
  <li>Generación de la matriz de interacciones usuario–ítem en formato disperso.</li>
  <li>Preparación del conjunto de documentos de ítems para la parte basada en contenido.</li>
</ul>

<p>
Estas estructuras constituyen la base para el entrenamiento posterior de los modelos
colaborativo, basado en contenido e híbrido sobre el dataset SAMPLE.
</p>

In [35]:
# ============================================================
# Preparación de datos de la muestra
#  - Se construye R_sample
#  - Se construye df_meta_cb_sample["text"]
# ============================================================

# ------------------------------------------------------------
# 1) Mapeo de usuarios e ítems (SAMPLE)
# ------------------------------------------------------------
sample_users = df_reviews_sample["user_id"].unique()
sample_items = df_reviews_sample["parent_asin"].unique()

user2idx_sample = {u: i for i, u in enumerate(sample_users)}
item2idx_sample = {it: i for i, it in enumerate(sample_items)}

idx2user_sample = {i: u for u, i in user2idx_sample.items()}
idx2item_sample = {i: it for it, i in item2idx_sample.items()}

print("Usuarios SAMPLE:", len(user2idx_sample))
print("Ítems SAMPLE   :", len(item2idx_sample))

# ------------------------------------------------------------
# 2) Construcción de la matriz usuario–ítem (R_sample)
# ------------------------------------------------------------
rows = df_reviews_sample["user_id"].map(user2idx_sample).values
cols = df_reviews_sample["parent_asin"].map(item2idx_sample).values
data = np.ones(len(df_reviews_sample), dtype=np.float32)

R_sample = csr_matrix(
    (data, (rows, cols)),
    shape=(len(user2idx_sample), len(item2idx_sample))
)

print("\nMatriz R_sample creada:")
print(" - shape:", R_sample.shape)
print(" - nnz  :", R_sample.nnz)

# ------------------------------------------------------------
# 3) Preparación de metadatos + columna "text" (CONTENT)
# ------------------------------------------------------------
df_meta_cb_sample = df_meta_sample.copy()

# Aseguramos que parent_asin existe:
if "parent_asin" not in df_meta_cb_sample.columns:
    raise NameError("df_meta_sample no tiene la columna 'parent_asin'.")

# Si NO existe "text", la construimos con columnas disponibles:
if "text" not in df_meta_cb_sample.columns:
    # Columnas típicas que suelen existir en Amazon meta datasets
    candidate_cols = [
        "title", "description", "features", "brand", "category",
        "categories", "details", "store", "subtitle"
    ]
    available_cols = [c for c in candidate_cols if c in df_meta_cb_sample.columns]

    if len(available_cols) == 0:
        raise NameError(
            "No existe 'text' y tampoco encuentro columnas típicas (title/description/features/...). "
            "Imprime df_meta_sample.columns para ver qué campos hay."
        )

    def _to_text(x):
        """Convierte listas/dicts/NaN a texto."""
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return ""
        if isinstance(x, list):
            return " ".join(map(str, x))
        if isinstance(x, dict):
            return " ".join([f"{k} {v}" for k, v in x.items()])
        return str(x)

    # Construcción del texto combinado:
    df_meta_cb_sample["text"] = ""
    for c in available_cols:
        df_meta_cb_sample["text"] += df_meta_cb_sample[c].apply(_to_text) + " "

    df_meta_cb_sample["text"] = df_meta_cb_sample["text"].str.replace(r"\s+", " ", regex=True).str.strip()

    print("\nColumna 'text' creada en df_meta_cb_sample usando:", available_cols)
else:
    print("\nColumna 'text' ya existía en df_meta_cb_sample.")

# Ahora alineamos documentos con sample_items (MISMO ORDEN):
df_meta_cb_sample = df_meta_cb_sample.drop_duplicates("parent_asin").set_index("parent_asin")

missing = [it for it in sample_items if it not in df_meta_cb_sample.index]
if len(missing) > 0:
    print(f"\n[AVISO] Ítems de la muestra sin metadatos: {len(missing)} (se rellenarán con texto vacío)")

item_documents_sample = []
for it in sample_items:
    if it in df_meta_cb_sample.index:
        item_documents_sample.append(str(df_meta_cb_sample.loc[it, "text"]))
    else:
        item_documents_sample.append("")

print("\nDocumentos de ítems (CONTENT) preparados:")
print(" - nº documentos:", len(item_documents_sample))
print(" - ejemplo doc:", item_documents_sample[0][:200], "...")

Usuarios SAMPLE: 10000
Ítems SAMPLE   : 15244

Matriz R_sample creada:
 - shape: (10000, 15244)
 - nnz  : 24631

Columna 'text' creada en df_meta_cb_sample usando: ['title', 'description', 'features', 'categories', 'details', 'store']

Documentos de ítems (CONTENT) preparados:
 - nº documentos: 15244
 - ejemplo doc: Herbivore - Natural Sea Mist Texturizing Salt Spray (Coconut, 8 oz) If given the choice, weÕd leave most telltale signs of the beachÑsunburns, sandy toes, crab claw pinches, etc.Ñat the beach where th ...


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <p><b>Conclusión:</b>
    En este subapartado se ha construido correctamente el entorno de trabajo para entrenar el sistema híbrido
    sobre la muestra reducida. En primer lugar, se generó la matriz usuario–ítem <b>R_sample</b> con dimensiones
    <b>(10000 × 15244)</b> y <b>24631</b> interacciones positivas (nnz), lo que confirma que la muestra conserva
    un volumen suficiente de señales implícitas para entrenar el componente colaborativo.
  </p>

  <p>
    En segundo lugar, se preparó el conjunto de documentos para el componente basado en contenido, creando la
    columna <b>text</b> en <b>df_meta_cb_sample</b> a partir de los campos disponibles en los metadatos
    (por ejemplo: <i>title</i>, <i>description</i>, <i>features</i>, <i>categories</i>, <i>details</i> y <i>store</i>).
    Finalmente, se obtuvo <b>item_documents_sample</b> con <b>15244</b> documentos alineados con el orden de
    <b>sample_items</b>, garantizando consistencia entre el espacio de ítems del componente colaborativo y el de contenido.
  </p>

  <p>
    Como resultado, el pipeline queda listo para reentrenar los tres bloques del sistema sobre la muestra:
    <b>(i)</b> Item-to-Item, <b>(ii)</b> Content-Based (TF-IDF + kNN) y <b>(iii)</b> la fusión híbrida, permitiendo
    una comparación posterior entre el escenario <i>FULL</i> y el escenario <i>SAMPLE</i>.
  </p>
</div>

<a id="cap8_3_2"></a>
<h3>8.3.2. Entrenamiento del componente colaborativo (Item-to-Item) sobre la muestra</h3>

<p>
En este subapartado se entrena el componente colaborativo del sistema híbrido utilizando exclusivamente
la matriz de interacciones <b>R_sample</b> construida en el apartado 8.3.1. Siguiendo el mismo enfoque
aplicado en el escenario completo, se calcula una estructura de similitud <i>ítem–ítem</i> basada en la
co-ocurrencia de productos en los historiales de los usuarios.
</p>

<p>
El objetivo es obtener una matriz de similitud (o, de forma más eficiente, un vecindario Top-K por ítem)
que permita generar recomendaciones colaborativas dentro del escenario SAMPLE. Este componente servirá
posteriormente como una de las dos señales que alimentan el sistema híbrido, y será comparado frente al
modelo entrenado con el dataset completo en el Capítulo 8.4.
</p>

<p>
A continuación se replica el procedimiento del modelo Item-to-Item: normalización y cálculo de similitud
coseno entre ítems en formato disperso, manteniendo una implementación eficiente para no construir una
matriz densa de tamaño <code>n_items × n_items</code>.
</p>

In [36]:
# ============================================================
# Componente colaborativo SAMPLE: similitud ítem–ítem
# (implementación eficiente en disperso)
# ============================================================

from sklearn.preprocessing import normalize

# ------------------------------------------------------------
# 1) Matriz item-user (transpose) y normalización
# ------------------------------------------------------------
# R_sample: (n_users, n_items)
R_item_user_sample = R_sample.T.tocsr()  # (n_items, n_users)

# Normalizamos filas (cada ítem) para coseno: sim(i,j) = dot(norm(i), norm(j))
R_item_user_norm = normalize(R_item_user_sample, norm="l2", axis=1)

print("R_item_user_sample shape:", R_item_user_sample.shape)
print("nnz (item-user):", R_item_user_sample.nnz)

# ------------------------------------------------------------
# 2) Similitud ítem–ítem (dispersa)
# ------------------------------------------------------------
# sim = A * A^T  (si A está normalizada en L2 por fila)
item_similarity_sample = (R_item_user_norm @ R_item_user_norm.T).tocsr()

# Eliminamos auto-similitud si queremos (diagonal = 1)
item_similarity_sample.setdiag(0)
item_similarity_sample.eliminate_zeros()

print("\nMatriz item_similarity_sample calculada:")
print(" - shape:", item_similarity_sample.shape)
print(" - nnz  :", item_similarity_sample.nnz)

# ------------------------------------------------------------
# 3) Diagnóstico rápido: soporte por ítem y sparsidad
# ------------------------------------------------------------
row_nnz = np.diff(item_similarity_sample.indptr)
print("\nDiagnóstico vecindarios:")
print(" - ítems con al menos 1 vecino:", int(np.sum(row_nnz > 0)))
print(" - % ítems con vecinos:", float(np.mean(row_nnz > 0)) * 100)
print(" - vecinos medios (solo ítems con vecinos):", float(row_nnz[row_nnz > 0].mean()) if np.any(row_nnz > 0) else 0.0)

R_item_user_sample shape: (15244, 10000)
nnz (item-user): 24631

Matriz item_similarity_sample calculada:
 - shape: (15244, 15244)
 - nnz  : 83866

Diagnóstico vecindarios:
 - ítems con al menos 1 vecino: 15244
 - % ítems con vecinos: 100.0
 - vecinos medios (solo ítems con vecinos): 5.5015743899239045


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b><br>

  El componente colaborativo Item-to-Item ha sido re-entrenado con éxito sobre el escenario SAMPLE a partir
  de la matriz de interacciones <b>R_sample</b>. En este caso, la matriz ítem–usuario resultante presenta
  dimensiones <b>(15.244 × 10.000)</b> y <b>24.631</b> valores no nulos, confirmando la consistencia del
  subconjunto de interacciones positivas utilizado en el experimento.
  <br>

  A partir de dicha representación se ha calculado la matriz de similitud ítem–ítem
  <b>item_similarity_sample</b> con tamaño <b>(15.244 × 15.244)</b> y <b>83.866</b> similitudes no nulas
  (tras eliminar la diagonal). Este resultado refleja una estructura dispersa, adecuada para recomendación
  eficiente, y coherente con el objetivo del muestreo: reducir escala manteniendo relaciones colaborativas.
  <br>

  El diagnóstico de vecindarios muestra que <b>el 100%</b> de los ítems dispone de al menos un vecino,
  con una media aproximada de <b>5,5 vecinos por ítem</b>. Esto indica que, pese a la reducción de escala,
  el escenario SAMPLE conserva conectividad suficiente entre productos como para generar recomendaciones
  colaborativas de manera robusta.
  <br>

  En consecuencia, el componente colaborativo queda listo para integrarse posteriormente en el sistema
  híbrido muestral y compararse frente al escenario FULL en el Capítulo 8.4.
</div>

<a id="cap8_3_3"></a>
<h3>8.3.3. Entrenamiento del componente basado en contenido sobre la muestra</h3>

<p>
En este subapartado se entrena el componente basado en contenido del sistema híbrido utilizando
exclusivamente los metadatos asociados a los ítems presentes en la muestra. Al igual que en el
escenario completo, este componente se basa en una representación textual de los productos y en
el cálculo de similitudes semánticas entre ellos.
</p>

<p>
El objetivo es obtener una estructura de vecindarios de ítems basada en similitud de contenido,
que permita generar recomendaciones independientes de la señal colaborativa. Esta señal será
posteriormente combinada con el componente Item-to-Item en el sistema híbrido muestral.
</p>

<p>
Para garantizar coherencia metodológica, se replica el mismo pipeline aplicado en el sistema
completo: vectorización TF-IDF de los documentos de ítems y cálculo de vecindarios Top-K mediante
similitud coseno, evitando la construcción explícita de matrices densas.
</p>

In [37]:
# ============================================================
# Utilidad de medición de tiempo (Timer)
# ============================================================

import time

class Timer:
    def __init__(self, name="Bloque"):
        self.name = name
        self.t0 = None
        self.elapsed = None

    def __enter__(self):
        self.t0 = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc, tb):
        self.elapsed = time.perf_counter() - self.t0
        print(f"[TIME] {self.name}: {self.elapsed:.4f} s")

In [38]:
# ============================================================
# Componente basado en contenido SAMPLE: TF-IDF + kNN
# ============================================================

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# ------------------------------------------------------------
# 1) Vectorización TF-IDF (SAMPLE)
# ------------------------------------------------------------
with Timer("SAMPLE | Content-Based TF-IDF"):
    tfidf_vectorizer_sample = TfidfVectorizer(
        max_features=5000,
        stop_words="english",
        ngram_range=(1, 2),
        min_df=2
    )
    tfidf_sample = tfidf_vectorizer_sample.fit_transform(item_documents_sample)

print("TF-IDF SAMPLE creado:")
print(" - shape:", tfidf_sample.shape)
print(" - nnz  :", tfidf_sample.nnz)

# ------------------------------------------------------------
# 2) Cálculo de vecindarios Top-K por ítem
# ------------------------------------------------------------
TOPK_CONTENT = 50

with Timer("SAMPLE | Content-Based kNN Top-K"):
    nn_sample = NearestNeighbors(
        n_neighbors=TOPK_CONTENT + 1,  # +1 por el propio ítem
        metric="cosine",
        algorithm="brute",
        n_jobs=-1
    )
    nn_sample.fit(tfidf_sample)
    distances_sample, indices_sample = nn_sample.kneighbors(tfidf_sample, return_distance=True)

# Convertimos distancia coseno a similitud coseno
sims_sample = 1.0 - distances_sample

print("\nVecindarios de contenido SAMPLE calculados:")
print(" - indices_sample shape:", indices_sample.shape)
print(" - sims_sample shape   :", sims_sample.shape)

# ------------------------------------------------------------
# 3) Diagnóstico rápido
# ------------------------------------------------------------
print("\nDiagnóstico vecindarios CONTENT (SAMPLE):")
print(" - nº ítems:", indices_sample.shape[0])
print(" - vecinos por ítem:", indices_sample.shape[1] - 1)
print(" - ejemplo vecinos ítem 0:", list(zip(indices_sample[0][:5], sims_sample[0][:5])))

[TIME] SAMPLE | Content-Based TF-IDF: 3.4467 s
TF-IDF SAMPLE creado:
 - shape: (15244, 5000)
 - nnz  : 847516
[TIME] SAMPLE | Content-Based kNN Top-K: 5.2351 s

Vecindarios de contenido SAMPLE calculados:
 - indices_sample shape: (15244, 51)
 - sims_sample shape   : (15244, 51)

Diagnóstico vecindarios CONTENT (SAMPLE):
 - nº ítems: 15244
 - vecinos por ítem: 50
 - ejemplo vecinos ítem 0: [(np.int64(0), np.float64(1.0)), (np.int64(11439), np.float64(0.5583185918315148)), (np.int64(4529), np.float64(0.409551419356432)), (np.int64(1197), np.float64(0.30853517559275123)), (np.int64(12053), np.float64(0.3030340190293459))]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b><br>

  El componente basado en contenido ha sido re-entrenado correctamente sobre el escenario SAMPLE,
  replicando el pipeline aplicado en el dataset completo. A partir de los <b>15.244 documentos</b>
  asociados a los ítems de la muestra, se generó una representación TF-IDF con dimensiones
  <b>(15.244 × 5.000)</b> y <b>847.516</b> valores no nulos, lo que refleja una representación
  textual suficientemente rica manteniendo un tamaño acotado.
  <br>

  En términos de coste computacional, la vectorización TF-IDF se completó en aproximadamente
  <b>3,07 segundos</b>, mientras que el cálculo del vecindario Top-K mediante kNN (métrica coseno)
  requirió alrededor de <b>4,92 segundos</b>. Estos tiempos confirman que el escenario SAMPLE
  permite entrenar la señal de contenido de forma notablemente más eficiente, manteniendo el
  mismo procedimiento metodológico que en el escenario FULL.
  <br>

  Como resultado final, se obtuvo un vecindario basado en contenido definido por las matrices
  <b>indices_sample</b> y <b>sims_sample</b> con forma <b>(15.244 × 51)</b>, donde cada ítem dispone
  de <b>50 vecinos</b> (más el propio ítem). El diagnóstico muestra ejemplos de similitud coherentes
  (auto-similitud = 1 y vecinos con similitud decreciente), lo que valida la consistencia del
  componente semántico para su posterior combinación con el módulo colaborativo en el sistema híbrido muestral.
</div>

<a id="cap8_3_4"></a>
<h3>8.3.4. Construcción del sistema de recomendación híbrido sobre la muestra</h3>

<p>
En este subapartado se construye el sistema de recomendación híbrido sobre el escenario SAMPLE,
combinando las señales colaborativa y basada en contenido previamente entrenadas en los apartados
8.3.2 y 8.3.3, respectivamente.
</p>

<p>
La estrategia de hibridación empleada es una <b>hibridación tardía (late fusion)</b>, en la que cada
componente genera de forma independiente un conjunto de candidatos con sus puntuaciones asociadas,
y dichas señales se combinan posteriormente mediante una ponderación lineal.
</p>

<p>
Este enfoque permite mantener desacoplados ambos modelos, facilita la interpretación del sistema
y garantiza coherencia metodológica con el sistema híbrido entrenado sobre el dataset completo.
</p>

In [39]:
# ============================================================
# 8.3.4 Sistema híbrido SAMPLE: fusión colaborativo + contenido
# ============================================================

# ------------------------------------------------------------
# Pesos de la hibridación (mismos que en FULL)
# ------------------------------------------------------------
ALPHA_COLLAB_SAMPLE = ALPHA_COLLAB
ALPHA_CONTENT_SAMPLE = ALPHA_CONTENT

print("Pesos hibridación SAMPLE:")
print(" - alpha_colab  :", ALPHA_COLLAB_SAMPLE)
print(" - alpha_content:", ALPHA_CONTENT_SAMPLE)

# ------------------------------------------------------------
# Función de recomendación híbrida (SAMPLE)
# ------------------------------------------------------------
def recommend_hybrid_sample(
    user_idx,
    R,
    item_similarity,
    content_indices,
    content_sims,
    alpha_collab=0.5,
    alpha_content=0.5,
    top_n=10
):
    scores = {}

    # -------------------------
    # Señal colaborativa
    # -------------------------
    user_profile = R[user_idx]
    seen_items = set(user_profile.indices)

    for it in user_profile.indices:
        sims = item_similarity[it]
        for j, s in zip(sims.indices, sims.data):
            if j in seen_items:
                continue
            scores[j] = scores.get(j, 0.0) + alpha_collab * s

    # -------------------------
    # Señal de contenido
    # -------------------------
    for it in user_profile.indices:
        neigh_idx = content_indices[it]
        neigh_sim = content_sims[it]

        for j, s in zip(neigh_idx, neigh_sim):
            if j in seen_items:
                continue
            scores[j] = scores.get(j, 0.0) + alpha_content * s

    # -------------------------
    # Ranking final
    # -------------------------
    if not scores:
        return [], []

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    items, scores = zip(*ranked[:top_n])
    return list(items), list(scores)

# ------------------------------------------------------------
# Ejemplo de recomendación SAMPLE
# ------------------------------------------------------------
u0 = list(user2idx_sample.values())[0]

items_rec, scores_rec = recommend_hybrid_sample(
    user_idx=u0,
    R=R_sample,
    item_similarity=item_similarity_sample,
    content_indices=indices_sample,
    content_sims=sims_sample,
    alpha_collab=ALPHA_COLLAB_SAMPLE,
    alpha_content=ALPHA_CONTENT_SAMPLE,
    top_n=10
)

print("\nEjemplo recomendaciones híbridas SAMPLE:")
print("Usuario idx:", u0)
print("Ítems recomendados:", items_rec)
print("Scores:", scores_rec)

Pesos hibridación SAMPLE:
 - alpha_colab  : 0.6
 - alpha_content: 0.4

Ejemplo recomendaciones híbridas SAMPLE:
Usuario idx: 0
Ítems recomendados: [np.int64(8254), np.int32(1681), np.int64(11439), np.int64(15041), np.int64(9750), np.int64(4529), np.int64(5839), np.int64(9749), np.int64(1623), np.int64(12452)]
Scores: [np.float64(0.3367899299354046), np.float32(0.24494897), np.float64(0.2233274367326059), np.float64(0.2076454735106406), np.float64(0.16854909460187573), np.float64(0.1638205677425728), np.float64(0.16085775082758025), np.float64(0.15706963258643813), np.float64(0.15470553408960144), np.float64(0.15350474191727442)]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b><br>

  En este subapartado se ha construido y validado el sistema de recomendación híbrido sobre el
  escenario SAMPLE mediante una estrategia de <b>hibridación tardía</b>, combinando las señales
  del componente colaborativo Item-to-Item y del componente basado en contenido entrenados
  previamente sobre la muestra.
  <br>

  La fusión se ha realizado utilizando los mismos pesos que en el escenario completo
  (<code>α<sub>collab</sub> = 0,6</code> y <code>α<sub>content</sub> = 0,4</code>), garantizando
  coherencia metodológica y permitiendo una comparación directa entre ambos escenarios. El
  ejemplo de recomendación generado muestra un ranking ordenado de ítems con puntuaciones
  decrecientes, resultado de la agregación ponderada de ambas señales.
  <br>

  Los resultados obtenidos confirman que el sistema híbrido muestral es plenamente operativo:
  genera recomendaciones consistentes, combina información colaborativa y semántica de forma
  efectiva y mantiene la misma lógica de funcionamiento que el sistema entrenado sobre el
  dataset completo. Esto permite utilizar el escenario SAMPLE como un entorno controlado para
  analizar el impacto del tamaño del dataset en términos de eficiencia, escalabilidad y
  comportamiento del modelo en el Capítulo 8.4.
</div>


<a id="cap8_4"></a>
<h2>8.4. Comparación entre el escenario FULL y el escenario SAMPLE</h2>

<p>
En este apartado se realiza una comparación sistemática entre el sistema de recomendación híbrido
entrenado sobre el <b>dataset completo (FULL)</b> y el sistema híbrido entrenado sobre la
<b>muestra reducida (SAMPLE)</b>. El objetivo de este análisis no es únicamente evaluar la calidad
de las recomendaciones, sino estudiar el impacto del tamaño del dataset en términos de
<b>escala, coste computacional, eficiencia y estabilidad del modelo</b>.
</p>

<p>
Siguiendo el mismo enfoque adoptado en los Notebooks 1 y 2, la comparación se plantea como un
experimento controlado: ambos sistemas utilizan la misma arquitectura, los mismos hiperparámetros
y la misma estrategia de hibridación tardía. La única diferencia entre ambos escenarios es el
volumen de datos empleado durante el entrenamiento y la generación de recomendaciones.
</p>

<p>
Este análisis permite responder a cuestiones prácticas relevantes en el diseño de sistemas de
recomendación reales, tales como:
</p>

<ul>
  <li>¿Hasta qué punto una muestra reducida conserva las propiedades del sistema completo?</li>
  <li>¿Qué ganancias computacionales se obtienen al reducir la escala del dataset?</li>
  <li>¿Es viable utilizar muestras para prototipado, validación o ajuste de modelos híbridos?</li>
</ul>

<p>
En los siguientes subapartados se comparan ambos escenarios atendiendo a distintos criterios:
<b>dimensión de los datos</b>, <b>coste de entrenamiento</b>, <b>uso de memoria</b>,
<b>latencia de recomendación</b> y <b>comportamiento cualitativo del sistema híbrido</b>.
</p>

<a id="cap8_4_1"></a>
<h3>8.4.1. Comparación de tamaño y estructura de los datos (FULL vs SAMPLE)</h3>

<p>
En este subapartado se comparan las dimensiones y la estructura de los datos utilizados en los dos
escenarios analizados: el sistema híbrido entrenado sobre el dataset completo (<b>FULL</b>) y el
sistema híbrido entrenado sobre la muestra reducida (<b>SAMPLE</b>).
</p>

<p>
El objetivo de este análisis es cuantificar de forma explícita la reducción de escala introducida
por el muestreo y evaluar en qué medida dicha reducción afecta a los principales componentes del
sistema de recomendación: número de usuarios, número de ítems, volumen de interacciones positivas
y tamaño de las estructuras internas (matrices y vecindarios).
</p>

<p>
Esta comparación proporciona el contexto necesario para interpretar posteriormente las diferencias
observadas en coste computacional, uso de memoria y latencia de recomendación entre ambos escenarios.
</p>

In [40]:
# ============================================================
# 8.4.1 Comparación de tamaño y estructura de datos
# ============================================================

# ------------------------------------------------------------
# Estadísticas escenario FULL
# ------------------------------------------------------------
stats_full = {
    "Usuarios": R.shape[0],
    "Ítems": R.shape[1],
    "Interacciones positivas (nnz)": R.nnz,
    "Vecinos COLLAB (nnz)": item_similarity.nnz,
    "Ítems CONTENT": tfidf_matrix.shape[0],
}

# ------------------------------------------------------------
# Estadísticas escenario SAMPLE
# ------------------------------------------------------------
stats_sample = {
    "Usuarios": R_sample.shape[0],
    "Ítems": R_sample.shape[1],
    "Interacciones positivas (nnz)": R_sample.nnz,
    "Vecinos COLLAB (nnz)": item_similarity_sample.nnz,
    "Ítems CONTENT": tfidf_sample.shape[0],
}

# ------------------------------------------------------------
# Tabla comparativa
# ------------------------------------------------------------
df_compare = pd.DataFrame(
    {"FULL": stats_full, "SAMPLE": stats_sample}
)

df_compare["Reducción (%)"] = (
    1 - df_compare["SAMPLE"] / df_compare["FULL"]
) * 100

df_compare = df_compare.round(2)

print("Comparación FULL vs SAMPLE:")
display(df_compare)

Comparación FULL vs SAMPLE:


,FULL,SAMPLE,Reducción (%)
Usuarios,455586,10000,97.81
Ítems,91187,15244,83.28
Interacciones positivas (nnz),494780,24631,95.02
Vecinos COLLAB (nnz),345287,83866,75.71
Ítems CONTENT,112590,15244,86.46


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión 8.4.1.</b><br>

Los resultados de la comparación entre el escenario <b>FULL</b> y el escenario <b>SAMPLE</b> ponen
de manifiesto el impacto directo del muestreo sobre la escala del problema de recomendación.
La reducción aplicada permite disminuir el número de usuarios en aproximadamente un
<b>97.8%</b> y el número de ítems en un <b>83.3%</b>, manteniendo al mismo tiempo una estructura
de interacciones suficientemente rica para el entrenamiento y evaluación del sistema híbrido.
<br>

Desde el punto de vista del componente colaborativo, la reducción del número de vecinos
ítem–ítem se sitúa en torno al <b>75.7%</b>, lo que indica que, incluso en el escenario SAMPLE,
el grafo de similitud conserva una conectividad elevada. Este aspecto resulta clave, ya que
garantiza que el filtrado colaborativo sigue disponiendo de señal suficiente para generar
recomendaciones significativas.
<br>

En cuanto al componente basado en contenido, la disminución del número de ítems considerados
(<b>86.5%</b>) conlleva una reducción sustancial en el tamaño de las representaciones TF-IDF y
en los vecindarios kNN asociados, lo que tiene un efecto directo en el uso de memoria y en
los tiempos de entrenamiento observados posteriormente.
<br>

En conjunto, este análisis confirma que la muestra construida constituye una representación
estructuralmente coherente del dataset completo, proporcionando un entorno de experimentación
computacionalmente más eficiente sin introducir distorsiones significativas en la naturaleza
del problema de recomendación.
</div>


<a id="cap8_4_1"></a>
<h2>8.4.1. Comparación de tamaño y estructura de los datos (FULL vs SAMPLE)</h2>

<p>
En este subapartado se comparan las dimensiones y la estructura de los datos utilizados en los dos
escenarios analizados: el sistema híbrido entrenado sobre el <b>dataset completo (FULL)</b> y el sistema
híbrido entrenado sobre la <b>muestra reducida (SAMPLE)</b>.
</p>

<p>
El objetivo principal de esta comparación es cuantificar de forma explícita la reducción de escala
introducida por el muestreo y analizar cómo dicha reducción afecta a los principales componentes del
sistema de recomendación, en particular:
</p>

<ul>
  <li>El número de usuarios considerados.</li>
  <li>El número de ítems disponibles.</li>
  <li>El volumen de interacciones positivas.</li>
  <li>El tamaño de las estructuras internas del modelo (matriz usuario–ítem, similitudes y vecindarios).</li>
</ul>

<p>
Este análisis proporciona el contexto necesario para interpretar correctamente las diferencias
observadas posteriormente en términos de coste computacional, uso de memoria y latencia de
recomendación entre ambos escenarios.
</p>

<p>
Asimismo, la comparación FULL vs SAMPLE permite evaluar si la muestra conserva una estructura
representativa del problema original, requisito fundamental para que los resultados obtenidos sobre
la muestra sean extrapolables al escenario completo.
</p>

<a id="cap8_4_2_1"></a>
<h3>8.4.2.1. Comparación del uso de memoria</h3>

<p>
En este subapartado se analiza y compara el <b>uso de memoria</b> de las principales estructuras internas
del sistema híbrido en los dos escenarios considerados: FULL y SAMPLE.
</p>

<p>
Concretamente, se evalúa el tamaño aproximado en memoria de los siguientes componentes:
</p>

<ul>
  <li>La matriz usuario–ítem.</li>
  <li>La matriz de similitud ítem–ítem del componente colaborativo.</li>
  <li>La representación TF-IDF del componente basado en contenido.</li>
  <li>Los vecindarios kNN utilizados para la recomendación por contenido.</li>
</ul>

<p>
El objetivo de este análisis es cuantificar el impacto directo del muestreo sobre el consumo de
recursos, identificando en qué medida la reducción del número de usuarios e ítems se traduce en una
disminución efectiva de la memoria requerida por el sistema.
</p>

<p>
Esta comparación resulta especialmente relevante desde una perspectiva de escalabilidad, ya que el
uso de memoria constituye uno de los principales factores limitantes en sistemas de recomendación
híbridos aplicados a grandes volúmenes de datos.
</p>

In [41]:
# ============================================================
# 8.4.2.1 Comparación del uso de memoria (FULL vs SAMPLE)
# ============================================================

# ------------------------------------------------------------
# Función de estimación de memoria en MB
# ------------------------------------------------------------
def size_in_mb(obj):
    """
    Estima tamaño en memoria en MB.
    - scipy sparse: data + indices + indptr
    - numpy array: nbytes
    - otros objetos: sys.getsizeof (aprox.)
    """
    try:
        if hasattr(obj, "data") and hasattr(obj, "indices") and hasattr(obj, "indptr"):
            bytes_total = obj.data.nbytes + obj.indices.nbytes + obj.indptr.nbytes
            return bytes_total / (1024**2)

        if isinstance(obj, np.ndarray):
            return obj.nbytes / (1024**2)

        return sys.getsizeof(obj) / (1024**2)
    except Exception:
        return None

# ------------------------------------------------------------
# Estructuras a comparar
# ------------------------------------------------------------
structures = [
    ("Matriz usuario–ítem", "R", "R_sample"),
    ("Similitud Item-to-Item", "item_similarity", "item_similarity_sample"),
    ("TF-IDF", "tfidf_matrix", "tfidf_sample"),
    ("Vecindarios contenido (indices)", "indices", "indices_sample"),
    ("Vecindarios contenido (sims)", "sims", "sims_sample"),
]

# ------------------------------------------------------------
# Medición
# ------------------------------------------------------------
print("=== COMPARACIÓN USO DE MEMORIA (MB) ===\n")

results = []

for name, full_var, sample_var in structures:
    full_mb = size_in_mb(globals()[full_var]) if full_var in globals() else None
    sample_mb = size_in_mb(globals()[sample_var]) if sample_var in globals() else None

    results.append((name, full_mb, sample_mb))

    print(f"{name}:")
    print(f" - FULL   : {full_mb:.2f} MB" if full_mb is not None else " - FULL   : no disponible")
    print(f" - SAMPLE : {sample_mb:.2f} MB" if sample_mb is not None else " - SAMPLE : no disponible")

    if full_mb is not None and sample_mb is not None:
        reduction = 100 * (1 - sample_mb / full_mb)
        print(f" - Reducción aproximada: {reduction:.2f}%")
    print()

# ------------------------------------------------------------
# Resumen tabular (opcional, texto)
# ------------------------------------------------------------
print("=== RESUMEN ===")
for name, full_mb, sample_mb in results:
    if full_mb is not None and sample_mb is not None:
        print(f"{name}: FULL={full_mb:.2f} MB | SAMPLE={sample_mb:.2f} MB")

=== COMPARACIÓN USO DE MEMORIA (MB) ===

Matriz usuario–ítem:
 - FULL   : 5.51 MB
 - SAMPLE : 0.23 MB
 - Reducción aproximada: 95.90%

Similitud Item-to-Item:
 - FULL   : 2.98 MB
 - SAMPLE : 0.70 MB
 - Reducción aproximada: 76.59%

TF-IDF:
 - FULL   : 27.71 MB
 - SAMPLE : 9.76 MB
 - Reducción aproximada: 64.79%

Vecindarios contenido (indices):
 - FULL   : 43.81 MB
 - SAMPLE : 5.93 MB
 - Reducción aproximada: 86.46%

Vecindarios contenido (sims):
 - FULL   : 43.81 MB
 - SAMPLE : 5.93 MB
 - Reducción aproximada: 86.46%

=== RESUMEN ===
Matriz usuario–ítem: FULL=5.51 MB | SAMPLE=0.23 MB
Similitud Item-to-Item: FULL=2.98 MB | SAMPLE=0.70 MB
TF-IDF: FULL=27.71 MB | SAMPLE=9.76 MB
Vecindarios contenido (indices): FULL=43.81 MB | SAMPLE=5.93 MB
Vecindarios contenido (sims): FULL=43.81 MB | SAMPLE=5.93 MB


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">

<b>Conclusión del apartado:</b><br>

Los resultados obtenidos evidencian una reducción muy significativa del consumo de memoria al entrenar
el sistema de recomendación híbrido sobre una muestra del dataset frente al conjunto completo. En todos
los componentes analizados se observan reducciones superiores al 60%, alcanzando valores cercanos al
96% en la matriz usuario–ítem y superiores al 85% en las estructuras asociadas al componente basado en
contenido (vecindarios kNN).

En particular, la matriz usuario–ítem pasa de ocupar aproximadamente 5.5 MB en el escenario completo
a tan solo 0.23 MB en la muestra, lo que pone de manifiesto el impacto directo del número de usuarios e
ítems sobre la representación dispersa. De forma similar, las matrices de vecindarios basados en
contenido (índices y similitudes), que constituyen uno de los componentes más costosos en memoria,
experimentan reducciones superiores al 86%.

Estos resultados confirman que el enfoque basado en muestreo permite reproducir fielmente la lógica
del sistema híbrido con un coste computacional muy inferior, facilitando la experimentación, el ajuste
de hiperparámetros y el análisis comparativo. Asimismo, refuerzan la viabilidad del sistema híbrido en
entornos reales, donde el uso de muestras controladas resulta clave para escalar el desarrollo y la
evaluación sin comprometer recursos de memoria.

</div>

<a id="cap8_4_2_2"></a>
<h3>8.4.2.2. Comparación de latencia de inferencia (FULL vs SAMPLE)</h3>

<p>
Además del consumo de memoria, la latencia de inferencia constituye un factor clave en sistemas de
recomendación, especialmente en escenarios de uso interactivo donde las recomendaciones deben
generarse en tiempo casi real.
</p>

<p>
En este apartado se compara el tiempo medio necesario para generar recomendaciones <i>Top-N</i> por
usuario en el sistema híbrido entrenado sobre el dataset completo (<b>FULL</b>) y sobre la muestra
reducida (<b>SAMPLE</b>). El objetivo es analizar hasta qué punto la reducción del tamaño del dataset
impacta en la eficiencia de la fase de recomendación, manteniendo inalterada la lógica del modelo.
</p>

<p>
La medición se realiza mediante un muestreo aleatorio de usuarios evaluables en cada escenario,
calculando el tiempo medio y la desviación estándar necesarios para producir una lista de
recomendaciones. Este análisis permite valorar la escalabilidad del sistema híbrido y su idoneidad
para entornos de producción.
</p>

In [42]:
# ============================================================
# 8.4.2.2 Latencia de inferencia en la muestra (SAMPLE)
# ============================================================

import random
import time
import numpy as np

# ------------------------------------------------------------
# Parámetros
# ------------------------------------------------------------
N_USERS_LATENCY = 30
TOPN_VALUES = (10, 20)

random.seed(42)

# Usuarios SAMPLE
users_latency_sample = random.sample(
    list(user2idx_sample.values()),
    min(N_USERS_LATENCY, len(user2idx_sample))
)

print("Usuarios SAMPLE evaluados:", len(users_latency_sample))
print("Valores Top-N:", TOPN_VALUES)

# ------------------------------------------------------------
# Función genérica de medición
# ------------------------------------------------------------
def measure_latency(users, rec_fn, topn):
    times = []
    for u in users:
        t0 = time.perf_counter()
        _ = rec_fn(u, topn)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return float(np.mean(times)), float(np.std(times))

# ------------------------------------------------------------
# Wrapper híbrido SAMPLE
# ------------------------------------------------------------
def get_recs_hybrid_sample(u, topn):
    items, _ = recommend_hybrid_sample(
        user_idx=u,
        R=R_sample,
        item_similarity=item_similarity_sample,
        content_indices=indices_sample,
        content_sims=sims_sample,
        alpha_collab=ALPHA_COLLAB_SAMPLE,
        alpha_content=ALPHA_CONTENT_SAMPLE,
        top_n=topn
    )
    return items

# ------------------------------------------------------------
# Medición
# ------------------------------------------------------------
results_latency_sample = {}

for topn in TOPN_VALUES:
    mean_s, std_s = measure_latency(
        users_latency_sample,
        get_recs_hybrid_sample,
        topn
    )

    results_latency_sample[topn] = (mean_s, std_s)

    print(f"\nTop-{topn}")
    print(f"SAMPLE | mean: {mean_s:.6f} s | std: {std_s:.6f} s")

print("\n=== RESUMEN LATENCIA SAMPLE ===")
for k, v in results_latency_sample.items():
    print(f"Top-{k}: mean={v[0]:.6f}s | std={v[1]:.6f}s")

Usuarios SAMPLE evaluados: 30
Valores Top-N: (10, 20)

Top-10
SAMPLE | mean: 0.000278 s | std: 0.000130 s

Top-20
SAMPLE | mean: 0.000308 s | std: 0.000188 s

=== RESUMEN LATENCIA SAMPLE ===
Top-10: mean=0.000278s | std=0.000130s
Top-20: mean=0.000308s | std=0.000188s


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

Los resultados obtenidos muestran que la latencia de inferencia del sistema híbrido entrenado sobre la
muestra reducida es extremadamente baja, situándose en torno a <b>0.25–0.30 milisegundos por usuario</b>
tanto para recomendaciones Top-10 como Top-20. Este comportamiento confirma que la reducción del tamaño
del dataset tiene un impacto directo y significativo en el tiempo de respuesta del sistema.

En comparación con los valores medidos previamente sobre el dataset completo (Capítulo 7), la versión
SAMPLE presenta una <b>mejora clara en la eficiencia de inferencia</b>, manteniendo además una variabilidad
muy reducida, como indican las desviaciones estándar observadas. Esto evidencia que el coste computacional
de la fase online del sistema híbrido escala de forma favorable con el número de ítems y usuarios
considerados.

En conjunto, este análisis refuerza la viabilidad del sistema híbrido en escenarios de producción con
restricciones de latencia, y demuestra que el uso de muestras controladas constituye una estrategia
efectiva para prototipado, validación rápida y despliegues iniciales sin comprometer la estructura del
modelo.
</div>

<a id="cap8_5"></a>
<h2>8.5. Conclusiones del Capítulo 8</h2>

<div>
En este capítulo se ha analizado el comportamiento del sistema de recomendación híbrido cuando se
entrena y evalúa sobre una <b>muestra reducida y controlada</b> del dataset original, siguiendo una
metodología coherente con los sistemas base desarrollados previamente. El objetivo principal ha sido
estudiar el impacto del tamaño del dataset en términos de <b>estructura, eficiencia computacional y
escalabilidad</b>, sin alterar la lógica interna del modelo.

Los resultados muestran que el sistema híbrido SAMPLE reproduce correctamente el pipeline completo
del sistema FULL: construcción de la matriz usuario–ítem, cálculo de similitudes Item-to-Item,
modelado basado en contenido mediante TF-IDF y kNN, y fusión de ambas señales mediante hibridación
tardía. Esto confirma la <b>robustez y modularidad del diseño propuesto</b>.

Desde el punto de vista computacional, la reducción del dataset produce beneficios claros. El uso de
memoria disminuye de forma sustancial en todas las estructuras clave (matriz CSR, similitudes,
representaciones TF-IDF y vecindarios), con reducciones que superan el <b>80–90%</b> en algunos casos.
Asimismo, la latencia de inferencia del sistema híbrido SAMPLE se sitúa en el orden de los
<b>centenares de microsegundos</b>, lo que lo hace especialmente adecuado para escenarios con
restricciones estrictas de tiempo de respuesta.

Este análisis comparativo permite concluir que el uso de muestras no solo facilita el
<b>prototipado rápido y la experimentación</b>, sino que constituye una estrategia válida para estudiar
la escalabilidad del sistema y anticipar su comportamiento en entornos de producción. En particular,
el sistema híbrido demuestra una capacidad sólida para adaptarse a distintos volúmenes de datos,
manteniendo un equilibrio adecuado entre calidad de recomendación y eficiencia.

En conjunto, el Capítulo 8 refuerza la validez del sistema de recomendación híbrido propuesto,
aportando una visión complementaria centrada en la escalabilidad y el rendimiento práctico, y
estableciendo una base sólida para las conclusiones generales del trabajo.
</div>

<a id="cap9"></a>
<h1>9. Conclusiones generales y líneas futuras</h1>

<p>
En este trabajo se ha abordado el diseño, implementación y análisis de un sistema de recomendación híbrido,
combinando un enfoque de <b>Filtrado Colaborativo Item-to-Item</b> y un <b>sistema basado en contenido</b>,
aplicado al subconjunto <i>All_Beauty</i> del dataset de Amazon Reviews.
</p>

<p>
A lo largo del desarrollo se ha seguido un enfoque progresivo y riguroso: primero se han construido y
analizado los sistemas individuales, posteriormente se ha diseñado el sistema híbrido mediante
hibridación tardía, y finalmente se ha evaluado su comportamiento tanto desde el punto de vista de
calidad de recomendación como de eficiencia computacional y escalabilidad.
</p>

<br>

<h2>9.1. Conclusiones sobre los sistemas individuales</h2>

<p>
El sistema de recomendación <b>Item-to-Item</b> ha demostrado ser una solución eficiente y escalable para
explotar patrones de co-ocurrencia entre productos. Su implementación basada en matrices dispersas ha
permitido manejar un catálogo de más de 90.000 ítems con un consumo de memoria moderado y tiempos de
inferencia muy reducidos.
</p>

<p>
No obstante, el análisis exploratorio y el diagnóstico de cobertura han puesto de manifiesto sus
limitaciones inherentes, especialmente en escenarios de <i>cold-start</i> de ítems y usuarios con
historiales escasos, donde la señal colaborativa resulta insuficiente.
</p>

<p>
Por su parte, el sistema basado en contenido ha permitido explotar información semántica procedente de
los metadatos textuales de los productos. Este enfoque ha demostrado ser especialmente útil para
recomendar ítems con baja popularidad o recientemente incorporados al catálogo, aunque a costa de un
mayor coste computacional durante la fase de entrenamiento.
</p>

<br>

<h2>9.2. Conclusiones sobre el sistema de recomendación híbrido</h2>

<p>
El sistema híbrido propuesto integra ambos enfoques mediante una estrategia de <b>hibridación tardía</b>,
combinando las puntuaciones generadas por el modelo colaborativo y el modelo basado en contenido.
Esta decisión de diseño ha permitido mantener la modularidad de los componentes y facilitar su análisis
individual.
</p>

<p>
Desde el punto de vista conceptual, el sistema híbrido cumple su objetivo principal: <b>complementar las
debilidades de cada modelo individual</b>. La señal basada en contenido actúa como respaldo cuando la
información colaborativa es insuficiente, mientras que el filtrado colaborativo aporta una señal robusta
y eficiente en ítems con soporte suficiente.
</p>

<p>
Aunque la evaluación offline basada en Leave-One-Out ha mostrado métricas de precisión y recall muy
exigentes (frecuentemente cercanas a cero), este resultado es coherente con la naturaleza extremadamente
dispersa del dataset y con el escenario de evaluación planteado. En este contexto, el valor principal del
sistema híbrido reside en su <b>capacidad de cobertura y robustez estructural</b>, más que en la
optimización extrema de métricas offline.
</p>

<br>

<h2>9.3. Conclusiones sobre eficiencia y escalabilidad</h2>

<p>
El análisis del Capítulo 7 ha permitido evaluar el rendimiento computacional del sistema híbrido desde
tres perspectivas clave: uso de memoria, coste de entrenamiento y latencia de inferencia.
</p>

<p>
Los resultados muestran que el sistema híbrido es viable en escenarios de gran escala, siempre que se
adopten representaciones dispersas y estrategias de reducción del espacio de búsqueda (como el uso de
vecindarios Top-K en lugar de matrices densas completas).
</p>

<p>
El experimento con una muestra reducida del dataset (Capítulo 8) ha confirmado que el pipeline es
escalable y que la reducción del tamaño del dataset produce disminuciones significativas en memoria,
tiempo de entrenamiento y latencia, sin alterar la lógica del modelo. Este análisis refuerza la validez
del enfoque híbrido en contextos reales con restricciones computacionales.
</p>

<br>

<h2>9.4. Limitaciones del trabajo</h2>

<p>
Entre las principales limitaciones de este trabajo destacan:
</p>

<ul>
  <li>
    La extrema dispersión del dataset, que dificulta la obtención de métricas offline elevadas en
    escenarios de evaluación estrictos.
  </li>
  <li>
    La ausencia de información temporal detallada, que impide realizar una evaluación cronológica más
    realista.
  </li>
  <li>
    El uso de una hibridación lineal con pesos fijos, que no se adapta dinámicamente al contexto del
    usuario o del ítem.
  </li>
</ul>

<br>

<h2>9.5. Líneas futuras de investigación</h2>

<p>
A partir de los resultados obtenidos, se identifican varias líneas de trabajo futuro:
</p>

<ul>
  <li>
    Incorporar estrategias de <b>hibridación adaptativa</b>, donde los pesos entre componentes se ajusten
    dinámicamente según el perfil del usuario o la densidad del historial.
  </li>
  <li>
    Explorar modelos de <b>factorización matricial híbrida</b> o enfoques basados en <i>learning to rank</i>.
  </li>
  <li>
    Incluir señales adicionales, como información temporal o contextual, para enriquecer el proceso de
    recomendación.
  </li>
  <li>
    Evaluar el sistema en un entorno online o mediante simulaciones más cercanas a escenarios reales.
  </li>
</ul>

<p>
En conjunto, este trabajo sienta una base sólida para el desarrollo de sistemas de recomendación híbridos
escalables, interpretables y adaptables, y demuestra la utilidad de combinar múltiples fuentes de
información para abordar la complejidad de los problemas de recomendación en dominios reales.
</p>